# ESTADISTICAS

In [1]:
import itertools
import tkinter as tk
from tkinter import ttk, messagebox
import csv
from tkcalendar import DateEntry  # Importa DateEntry
from tkinter import PhotoImage
import sys
import os

def resource_path(relative_path):
    """Obtener la ruta absoluta al recurso para ejecutables empaquetados."""
    try:
        # Si está empaquetado
        base_path = sys._MEIPASS
    except AttributeError:
        # Si no está empaquetado
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)

archivo_resultados = resource_path("resultados.csv")

# Participantes
jugadores = ["Ibai", "Xabi", "Ian", "Aitor", "Cifu", "David", "Igarki", "Aimar", "Erli", "Maria", "Dani", "AnderM", "Abad", "Sanchez"]

# Generar todas las combinaciones posibles de equipos
parejas = list(itertools.combinations(jugadores, 2))

# Crear un diccionario para mapear el string del equipo a los jugadores
equipos_str = ["{} & {}".format(j1, j2) for (j1, j2) in parejas]
equipo_str_a_pareja = dict(zip(equipos_str, parejas))

resultados = []  # Aquí se guardan todos los partidos que se leen o se registran

def leer_resultados():
    if os.path.exists(archivo_resultados):
        with open(archivo_resultados, mode='r', newline='', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    equipo1_jugador1 = row.get("equipo1_jugador1", "").strip() or "Desconocido"
                    equipo1_jugador2 = row.get("equipo1_jugador2", "").strip() or "Desconocido"
                    equipo2_jugador1 = row.get("equipo2_jugador1", "").strip() or "Desconocido"
                    equipo2_jugador2 = row.get("equipo2_jugador2", "").strip() or "Desconocido"
                    ganador_primer_set_jugador1 = row.get("ganador_primer_set_jugador1", "").strip() or "Desconocido"
                    ganador_primer_set_jugador2 = row.get("ganador_primer_set_jugador2", "").strip() or "Desconocido"
                    ganador_partido_jugador1 = row.get("ganador_partido_jugador1", "").strip() or "Desconocido"
                    ganador_partido_jugador2 = row.get("ganador_partido_jugador2", "").strip() or "Desconocido"
                    puntuaciones = row.get("puntuaciones", "").strip().split(';') if row.get("puntuaciones") else []

                    resultado = {
                        "partido": (
                            (equipo1_jugador1, equipo1_jugador2),
                            (equipo2_jugador1, equipo2_jugador2)
                        ),
                        "ganador_primer_set": (ganador_primer_set_jugador1, ganador_primer_set_jugador2),
                        "ganador_partido": (ganador_partido_jugador1, ganador_partido_jugador2),
                        "mvp": row.get("mvp", "Desconocido").strip(),
                        "puntuaciones": puntuaciones,
                        "tie_breaks": int(row["tie_breaks"]) if row.get("tie_breaks") else 0,
                        "lugar": row.get("lugar", "Desconocido").strip(),
                        "fecha": row.get("fecha", "Desconocida").strip()
                    }
                    resultados.append(resultado)
                except Exception as e:
                    print(f"Error procesando fila: {row}, Error: {e}")

def guardar_resultado_csv(resultado):
    file_exists = os.path.exists(archivo_resultados)
    with open(archivo_resultados, mode='a', newline='', encoding='utf-8-sig') as file:
        fieldnames = [
            "equipo1_jugador1", "equipo1_jugador2",
            "equipo2_jugador1", "equipo2_jugador2",
            "ganador_primer_set_jugador1", "ganador_primer_set_jugador2",
            "ganador_partido_jugador1", "ganador_partido_jugador2",
            "mvp", "puntuaciones", "tie_breaks", "lugar", "fecha"
        ]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            "equipo1_jugador1": resultado["partido"][0][0],
            "equipo1_jugador2": resultado["partido"][0][1],
            "equipo2_jugador1": resultado["partido"][1][0],
            "equipo2_jugador2": resultado["partido"][1][1],
            "ganador_primer_set_jugador1": resultado["ganador_primer_set"][0],
            "ganador_primer_set_jugador2": resultado["ganador_primer_set"][1],
            "ganador_partido_jugador1": resultado["ganador_partido"][0],
            "ganador_partido_jugador2": resultado["ganador_partido"][1],
            "mvp": resultado["mvp"],
            "puntuaciones": ';'.join(resultado["puntuaciones"]),
            "tie_breaks": resultado["tie_breaks"],
            "lugar": resultado["lugar"],
            "fecha": resultado["fecha"]
        })

def calcular_estadisticas(resultados_filtrar):
    lugares = ["Ibaiondo", "Bakh", "Otro"]
    estadisticas = {}
    for jugador in jugadores:
        estadisticas[jugador] = {
            "partidos_jugados": 0,
            "victorias": 0,
            "mvp": 0,
            "sets_jugados": 0,
            "sets_ganados": 0,
            "tie_breaks": 0,
            "primer_set_ganado": 0,
            "games_ganados": 0,
            "games_perdidos": 0,
            "victorias_por_lugar": {lugar: 0 for lugar in lugares}
        }

    for resultado in resultados_filtrar:
        equipo1, equipo2 = resultado["partido"]
        ganador = resultado["ganador_partido"]
        mvp = resultado["mvp"]
        lugar = resultado["lugar"]
        sets_jugados = len(resultado["puntuaciones"])

        # Asegurarnos de que cualquier jugador inesperado aparezca en el diccionario
        for jugador in equipo1 + equipo2:
            if jugador not in estadisticas:
                estadisticas[jugador] = {
                    "partidos_jugados": 0,
                    "victorias": 0,
                    "mvp": 0,
                    "sets_jugados": 0,
                    "sets_ganados": 0,
                    "tie_breaks": 0,
                    "primer_set_ganado": 0,
                    "games_ganados": 0,
                    "games_perdidos": 0,
                    "victorias_por_lugar": {lugar: 0 for lugar in lugares}
                }

        # Partidos jugados y sets jugados
        for jugador in equipo1 + equipo2:
            estadisticas[jugador]["partidos_jugados"] += 1
            estadisticas[jugador]["sets_jugados"] += sets_jugados

        # Victoria del partido
        for jugador in ganador:
            estadisticas[jugador]["victorias"] += 1
            if lugar in estadisticas[jugador]["victorias_por_lugar"]:
                estadisticas[jugador]["victorias_por_lugar"][lugar] += 1

        # Ganador del primer set
        for jugador in resultado["ganador_primer_set"]:
            estadisticas[jugador]["primer_set_ganado"] += 1

        # Sets y games ganados/perdidos
        for set_result in resultado["puntuaciones"]:
            if '(' in set_result:
                score_part, tiebreak_part = set_result.split('(')
                score1, score2 = map(int, score_part.split('-'))
                tie_breaks_in_set = 1
            else:
                score1, score2 = map(int, set_result.split('-'))
                tie_breaks_in_set = 0

            for jugador in equipo1 + equipo2:
                estadisticas[jugador]["tie_breaks"] += tie_breaks_in_set

            # Set ganado
            if score1 > score2:
                for jugador in equipo1:
                    estadisticas[jugador]["sets_ganados"] += 1
            else:
                for jugador in equipo2:
                    estadisticas[jugador]["sets_ganados"] += 1

            # Games
            for jugador in equipo1:
                estadisticas[jugador]["games_ganados"] += score1
                estadisticas[jugador]["games_perdidos"] += score2
            for jugador in equipo2:
                estadisticas[jugador]["games_ganados"] += score2
                estadisticas[jugador]["games_perdidos"] += score1

        # MVP
        if mvp in estadisticas:
            estadisticas[mvp]["mvp"] += 1

    # Calcular porcentajes y diferencias
    for jugador, stats in estadisticas.items():
        pj = stats["partidos_jugados"]
        if pj > 0:
            stats["porcentaje_victorias"] = stats["victorias"] / pj * 100
            stats["porcentaje_primer_set"] = stats["primer_set_ganado"] / pj * 100
        else:
            stats["porcentaje_victorias"] = 0
            stats["porcentaje_primer_set"] = 0
        stats["diferencia_games"] = stats["games_ganados"] - stats["games_perdidos"]

    return estadisticas

def generar_ranking(estadisticas):
    ranking = [(jugador, stats) for jugador, stats in estadisticas.items()]
    ranking.sort(key=lambda x: x[1]["porcentaje_victorias"], reverse=True)
    return ranking

def mostrar_ranking():
    estadisticas = calcular_estadisticas(resultados)

    años = sorted({r["fecha"][:4] for r in resultados})
    años.insert(0, "Todos")

    ranking_window = tk.Toplevel()
    ranking_window.title("Ranking de Jugadores")

    style = ttk.Style(ranking_window)
    style.theme_use('clam')
    style.configure('Treeview', background='#E3F2FD', foreground='black', rowheight=25, fieldbackground='#E3F2FD')
    style.configure('Treeview.Heading', background='#1E88E5', foreground='white', font=('Helvetica', 10, 'bold'))

    table_frame = tk.Frame(ranking_window)
    table_frame.pack(fill='both', expand=True)

    filtro_año_var = tk.StringVar(value="Todos")
    tk.Label(ranking_window, text="Filtrar por Año:", font=('Helvetica', 12)).pack(pady=5)
    filtro_año_combobox = ttk.Combobox(ranking_window, textvariable=filtro_año_var, values=años, state='readonly')
    filtro_año_combobox.pack(pady=5)

    tree_scroll_y = ttk.Scrollbar(table_frame, orient='vertical')
    tree_scroll_y.pack(side='right', fill='y')

    tree_scroll_x = ttk.Scrollbar(table_frame, orient='horizontal')
    tree_scroll_x.pack(side='bottom', fill='x')

    columns = (
        "Posición", "Jugador", "Porc. Victorias", "Porc. 1er Set", "Sets Jugados",
        "Victorias", "Partidos Jugados", "Tie Breaks", "MVPs", "Games Ganados",
        "Games Perdidos", "Dif. Games", "Victorias en Ibaiondo", "Victorias en Bakh", "Victorias en Otro"
    )
    tree = ttk.Treeview(table_frame, columns=columns, show='headings',
                        yscrollcommand=tree_scroll_y.set, xscrollcommand=tree_scroll_x.set)
    tree_scroll_y.config(command=tree.yview)
    tree_scroll_x.config(command=tree.xview)

    column_widths = {
        "Posición": 60, "Jugador": 100, "Porc. Victorias": 100, "Porc. 1er Set": 100,
        "Sets Jugados": 90, "Victorias": 70, "Partidos Jugados": 110, "Tie Breaks": 80,
        "MVPs": 60, "Games Ganados": 100, "Games Perdidos": 100, "Dif. Games": 80,
        "Victorias en Ibaiondo": 130, "Victorias en Bakh": 120, "Victorias en Otro": 110
    }

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='center', width=column_widths.get(col, 100), stretch=False)

    tree.pack(expand=True, fill='both')

    def actualizar_ranking():
        año_seleccionado = filtro_año_var.get()
        if año_seleccionado == "Todos":
            resultados_filtrados = resultados
        else:
            resultados_filtrados = [r for r in resultados if r["fecha"].startswith(año_seleccionado)]

        est_filtradas = calcular_estadisticas(resultados_filtrados)
        ranking_filtrado = generar_ranking(est_filtradas)

        for item in tree.get_children():
            tree.delete(item)

        posicion = 1
        for jugador, stats in ranking_filtrado:
            valores = [
                posicion,
                jugador,
                f"{stats['porcentaje_victorias']:.2f}%",
                f"{stats['porcentaje_primer_set']:.2f}%",
                stats['sets_jugados'],
                stats['victorias'],
                stats['partidos_jugados'],
                stats['tie_breaks'],
                stats['mvp'],
                stats['games_ganados'],
                stats['games_perdidos'],
                stats['diferencia_games'],
                stats['victorias_por_lugar'].get("Ibaiondo", 0),
                stats['victorias_por_lugar'].get("Bakh", 0),
                stats['victorias_por_lugar'].get("Otro", 0)
            ]
            tree.insert('', tk.END, values=valores)
            posicion += 1

    filtro_año_combobox.bind("<<ComboboxSelected>>", lambda e: actualizar_ranking())
    actualizar_ranking()

    ttk.Button(ranking_window, text="Cerrar", command=ranking_window.destroy).pack(pady=10)

    # Ordenar columnas al hacer clic
    tree_sort_column = {'column': None, 'descending': False}
    def sort_treeview(treeview, col):
        data = []
        for child in treeview.get_children():
            item = treeview.item(child)
            values = list(item['values'])
            data.append((values, child))

        col_indices = {columns[i]: i for i in range(len(columns))}
        col_index = col_indices[col]

        if tree_sort_column['column'] == col:
            descending = not tree_sort_column['descending']
        else:
            descending = False
        tree_sort_column['column'] = col
        tree_sort_column['descending'] = descending

        def convert(value):
            try:
                if isinstance(value, str) and value.endswith('%'):
                    return float(value.rstrip('%'))
                return float(value)
            except ValueError:
                return value.lower() if isinstance(value, str) else value

        data.sort(key=lambda x: convert(x[0][col_index]), reverse=descending)

        for index, (values, item) in enumerate(data):
            values[0] = index + 1
            treeview.item(item, values=values)
            treeview.move(item, '', index)

    for col in columns:
        tree.heading(col, text=col, command=lambda _col=col: sort_treeview(tree, _col))


# --- INICIO CAMBIOS: Función para MOSTRAR PARTIDOS con filtro por jugador ---
def mostrar_partidos():
    """Abre una ventana con todos los partidos y permite filtrar por un jugador."""
    partidos_window = tk.Toplevel()
    partidos_window.title("Lista de Partidos")

    style = ttk.Style(partidos_window)
    style.theme_use('clam')
    style.configure('Treeview', background='#E3F2FD', foreground='black', rowheight=25, fieldbackground='#E3F2FD')
    style.configure('Treeview.Heading', background='#1E88E5', foreground='white', font=('Helvetica', 10, 'bold'))

    filtro_frame = tk.Frame(partidos_window)
    filtro_frame.pack(pady=5)

    tk.Label(filtro_frame, text="Filtrar por Jugador:", font=('Helvetica', 12)).grid(row=0, column=0, padx=5)
    jugador_filtro_var = tk.StringVar(value="Todos")
    # Añadimos "Todos" como opción adicional
    lista_jugadores_filtro = ["Todos"] + jugadores
    jugador_filtro_combobox = ttk.Combobox(filtro_frame, textvariable=jugador_filtro_var,
                                           values=lista_jugadores_filtro, state='readonly')
    jugador_filtro_combobox.grid(row=0, column=1, padx=5)

    # Frame y scrollbars para el Treeview
    table_frame = tk.Frame(partidos_window)
    table_frame.pack(fill='both', expand=True)

    scroll_y = ttk.Scrollbar(table_frame, orient='vertical')
    scroll_y.pack(side='right', fill='y')

    scroll_x = ttk.Scrollbar(table_frame, orient='horizontal')
    scroll_x.pack(side='bottom', fill='x')

    columnas = ["Fecha", "Equipo 1", "Equipo 2", "Puntuaciones", "Ganador", "MVP", "Tie-breaks", "Lugar"]
    tree_partidos = ttk.Treeview(table_frame, columns=columnas, show='headings',
                                 yscrollcommand=scroll_y.set, xscrollcommand=scroll_x.set)
    scroll_y.config(command=tree_partidos.yview)
    scroll_x.config(command=tree_partidos.xview)

    # Configurar encabezados de columnas
    for col in columnas:
        tree_partidos.heading(col, text=col)
        tree_partidos.column(col, anchor='center', width=120, stretch=False)

    tree_partidos.pack(expand=True, fill='both')

    def actualizar_partidos():
        """Filtra los partidos según el jugador y actualiza el Treeview."""
        tree_partidos.delete(*tree_partidos.get_children())  # Limpiar
        jugador_seleccionado = jugador_filtro_var.get()

        for r in resultados:
            # Extraemos datos:
            fecha = r["fecha"]
            eq1_str = " & ".join(r["partido"][0])
            eq2_str = " & ".join(r["partido"][1])
            puntuaciones = "; ".join(r["puntuaciones"]) if r["puntuaciones"] else "N/A"
            ganador_partido = " & ".join(r["ganador_partido"])
            mvp = r["mvp"]
            tie_breaks = r["tie_breaks"]
            lugar = r["lugar"]

            # Lógica de filtrado:
            # Si "Todos", mostramos todo
            # Si no, solo si el jugador está en equipo1 o en equipo2
            if jugador_seleccionado == "Todos":
                mostrar = True
            else:
                jugadores_partido = r["partido"][0] + r["partido"][1]
                mostrar = (jugador_seleccionado in jugadores_partido)

            if mostrar:
                tree_partidos.insert('', tk.END, values=(
                    fecha, eq1_str, eq2_str, puntuaciones, ganador_partido, mvp, tie_breaks, lugar
                ))

    # Botón para aplicar el filtro
    boton_filtrar = tk.Button(filtro_frame, text="Filtrar", command=actualizar_partidos, bg='#1E88E5', fg='white')
    boton_filtrar.grid(row=0, column=2, padx=5)

    # Cerrar la ventana
    ttk.Button(partidos_window, text="Cerrar", command=partidos_window.destroy).pack(pady=5)

    # Primero mostramos todos
    actualizar_partidos()

# --- FIN CAMBIOS ---


def crear_interfaz():
    root = tk.Tk()
    root.title("Registrar Resultado de Partido")

    style = ttk.Style(root)
    style.theme_use('clam')
    primary_color = '#1E88E5'
    background_color = '#E3F2FD'
    root.configure(bg=background_color)

    set_resultados = {}
    tie_break_vars = {}
    tie_break_scores = {}

    def toggle_tiebreak_entry(set_num):
        if tie_break_vars[set_num].get():
            tie_break_scores[set_num].config(state='normal')
        else:
            tie_break_scores[set_num].delete(0, tk.END)
            tie_break_scores[set_num].config(state='disabled')

    def registrar():
        equipo1_str = equipo1_var.get()
        equipo2_str = equipo2_var.get()
        fecha = fecha_var.get_date()

        if not equipo1_str or not equipo2_str:
            messagebox.showerror("Error", "Debes seleccionar ambos equipos.")
            return
        if equipo1_str == equipo2_str:
            messagebox.showerror("Error", "Los equipos no pueden ser iguales.")
            return

        equipo1 = equipo_str_a_pareja[equipo1_str]
        equipo2 = equipo_str_a_pareja[equipo2_str]

        ganador_primer_set_str = ganador_primer_set_var.get()
        ganador_partido_str = ganador_partido_var.get()
        if not ganador_primer_set_str or not ganador_partido_str:
            messagebox.showerror("Error", "Debes seleccionar los equipos ganadores.")
            return

        ganador_primer_set = equipo1 if ganador_primer_set_str == equipo1_str else equipo2
        ganador_partido = equipo1 if ganador_partido_str == equipo1_str else equipo2

        mvp = mvp_jugador.get()
        if not mvp:
            messagebox.showerror("Error", "Debes seleccionar el MVP.")
            return

        puntuaciones = []
        tie_breaks = 0
        for set_num in range(1, 4):
            set_resultado = set_resultados[set_num].get()
            tie_break_var = tie_break_vars[set_num]
            tie_break_score = tie_break_scores[set_num].get()

            if set_resultado:
                try:
                    score1, score2 = map(int, set_resultado.split('-'))
                except ValueError:
                    messagebox.showerror("Error", f"El resultado del Set {set_num} debe ser 'n-n'.")
                    return
                if tie_break_var.get():
                    if not tie_break_score:
                        messagebox.showerror("Error", f"Debes ingresar la puntuación del Tie-break para el Set {set_num}.")
                        return
                    puntuacion = f"{set_resultado}({tie_break_score})"
                    tie_breaks += 1
                else:
                    puntuacion = set_resultado
                puntuaciones.append(puntuacion)

        lugar = lugar_var.get()
           resultado = {
            "partido": (equipo1, equipo2),
            "ganador_primer_set": ganador_primer_set,
            "ganador_partido": ganador_partido,
            "mvp": mvp,
            "puntuaciones": puntuaciones,
            "tie_breaks": tie_breaks,
            "lugar": lugar,
            "fecha": fecha.strftime('%Y-%m-%d')
        }
    
    # Actualizamos Elo y capturamos los cambios
    cambios = actualizar_elo(resultado)
    resultado["elo_changes"] = cambios  # Se agrega al registro

    resultados.append(resultado)
    guardar_resultado_csv(resultado)  # Si decides incluir este campo en el CSV, actualiza también fieldnames
    messagebox.showinfo("Éxito", "Resultado registrado exitosamente")
        
        resultados.append(resultado)
        guardar_resultado_csv(resultado)
        messagebox.showinfo("Éxito", "Resultado registrado exitosamente")

        # Limpiar campos
        equipo1_var.set("")
        equipo2_var.set("")
        ganador_primer_set_var.set("")
        ganador_partido_var.set("")
        mvp_jugador.set("")
        for set_num in range(1, 4):
            set_resultados[set_num].delete(0, tk.END)
            tie_break_vars[set_num].set(False)
            tie_break_scores[set_num].delete(0, tk.END)
            tie_break_scores[set_num].config(state='disabled')
        lugar_var.set("Ibaiondo")

        ganador_primer_set_combobox['values'] = []
        ganador_partido_combobox['values'] = []

        actualizar_matriz_resultados()

    def actualizar_ganadores(*args):
        equipo1_str = equipo1_var.get()
        equipo2_str = equipo2_var.get()
        if equipo1_str and equipo2_str:
            opciones_ganadores = [equipo1_str, equipo2_str]
            ganador_primer_set_combobox['values'] = opciones_ganadores
            ganador_partido_combobox['values'] = opciones_ganadores
        else:
            ganador_primer_set_combobox['values'] = []
            ganador_partido_combobox['values'] = []

    def actualizar_matriz_resultados():
        for widget in matriz_resultados_frame_inner.winfo_children():
            widget.destroy()

        encabezados = ["Equipo 1", "Equipo 2", "Ganador 1er Set", "Ganador Partido", "MVP", "Puntuaciones", "Tie Breaks", "Lugar", "Fecha"]
        for i, resultado in enumerate(resultados, start=1):
            try:
                eq1 = " & ".join(resultado["partido"][0])
                eq2 = " & ".join(resultado["partido"][1])
                g1set = " & ".join(resultado["ganador_primer_set"])
                gpart = " & ".join(resultado["ganador_partido"])
                mvp = resultado["mvp"]
                punts = "; ".join(resultado["puntuaciones"])
                tb = resultado["tie_breaks"]
                lugar = resultado["lugar"]
                fecha = resultado["fecha"]

                datos = [eq1, eq2, g1set, gpart, mvp, punts, tb, lugar, fecha]
                for idx, dato in enumerate(datos):
                    label = tk.Label(matriz_resultados_frame_inner, text=dato, bg=background_color)
                    label.grid(row=i, column=idx, padx=5, pady=5)
                    label.configure(borderwidth=1, relief="solid")
            except Exception as e:
                print(f"Error mostrando resultado: {resultado}, Error: {e}")

        matriz_resultados_canvas.configure(scrollregion=matriz_resultados_canvas.bbox("all"))

    tk.Label(root, text="Fecha del Partido:", font=('Helvetica', 12), bg=background_color).grid(row=0, column=0, sticky='e')
    fecha_var = DateEntry(root, width=12, background='darkblue', foreground='white', borderwidth=2, date_pattern='y-mm-dd')
    fecha_var.grid(row=0, column=1, pady=5, padx=5)

    tk.Label(root, text="Equipo 1:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=1, column=0, sticky='e')
    equipo1_var = tk.StringVar()
    equipo1_combobox = ttk.Combobox(root, textvariable=equipo1_var, values=equipos_str)
    equipo1_combobox.grid(row=1, column=1, pady=5, padx=5)

    tk.Label(root, text="Equipo 2:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=2, column=0, sticky='e')
    equipo2_var = tk.StringVar()
    equipo2_combobox = ttk.Combobox(root, textvariable=equipo2_var, values=equipos_str)
    equipo2_combobox.grid(row=2, column=1, pady=5, padx=5)

    equipo1_var.trace('w', actualizar_ganadores)
    equipo2_var.trace('w', actualizar_ganadores)

    tk.Label(root, text="Ganador 1er Set:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=3, column=0, sticky='e')
    ganador_primer_set_var = tk.StringVar()
    ganador_primer_set_combobox = ttk.Combobox(root, textvariable=ganador_primer_set_var)
    ganador_primer_set_combobox.grid(row=3, column=1, pady=5, padx=5)

    tk.Label(root, text="Ganador Partido:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=4, column=0, sticky='e')
    ganador_partido_var = tk.StringVar()
    ganador_partido_combobox = ttk.Combobox(root, textvariable=ganador_partido_var)
    ganador_partido_combobox.grid(row=4, column=1, pady=5, padx=5)

    tk.Label(root, text="MVP:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=5, column=0, sticky='e')
    mvp_jugador = ttk.Combobox(root, values=jugadores)
    mvp_jugador.grid(row=5, column=1, pady=5, padx=5)

    for set_num in range(1, 4):
        tk.Label(root, text=f"Set {set_num} (ej: 6-4):", font=('Helvetica', 12), bg=background_color).grid(row=5 + set_num, column=0, sticky='e')
        set_resultados[set_num] = ttk.Entry(root)
        set_resultados[set_num].grid(row=5 + set_num, column=1, pady=5, padx=5)

        tie_break_vars[set_num] = tk.BooleanVar()
        tie_break_cb = tk.Checkbutton(root, text="Tie-break", variable=tie_break_vars[set_num], bg=background_color,
                                      command=lambda num=set_num: toggle_tiebreak_entry(num))
        tie_break_cb.grid(row=5 + set_num, column=2, padx=5)

        tk.Label(root, text=f"Puntuación Tie-break Set {set_num}:", font=('Helvetica', 12), bg=background_color).grid(row=5 + set_num, column=3, sticky='e')
        tie_break_scores[set_num] = ttk.Entry(root, state='disabled')
        tie_break_scores[set_num].grid(row=5 + set_num, column=4, pady=5, padx=5)

    tk.Label(root, text="Lugar del Partido:", font=('Helvetica', 12), bg=background_color).grid(row=9, column=0, sticky='e')
    lugar_var = tk.StringVar(value="Ibaiondo")
    lugar_options = ["Ibaiondo", "Bakh", "Otro"]
    lugar_menu = ttk.Combobox(root, textvariable=lugar_var, values=lugar_options)
    lugar_menu.grid(row=9, column=1, pady=5, padx=5)

    botones_frame = tk.Frame(root, bg=background_color)
    botones_frame.grid(row=10, columnspan=5, pady=10)

    registrar_button = tk.Button(botones_frame, text="Registrar Resultado", command=registrar,
                                 bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
    registrar_button.grid(row=0, column=0, padx=5)

    ranking_button = tk.Button(botones_frame, text="Mostrar Ranking", command=mostrar_ranking,
                               bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
    ranking_button.grid(row=0, column=1, padx=5)

    # --- INICIO CAMBIOS: Botón nuevo para MOSTRAR PARTIDOS ---
    partidos_button = tk.Button(botones_frame, text="Mostrar Partidos", command=mostrar_partidos,
                                bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
    partidos_button.grid(row=0, column=2, padx=5)
    # --- FIN CAMBIOS ---

    resultados_frame = tk.Frame(root, bg=background_color)
    resultados_frame.grid(row=11, columnspan=5, pady=10, sticky='nsew')

    global matriz_resultados_canvas
    matriz_resultados_canvas = tk.Canvas(resultados_frame, bg=background_color)
    matriz_resultados_canvas.grid(row=0, column=0, sticky='nsew')

    scrollbar = ttk.Scrollbar(resultados_frame, orient="vertical", command=matriz_resultados_canvas.yview)
    scrollbar.grid(row=0, column=1, sticky='ns')
    matriz_resultados_canvas.configure(yscrollcommand=scrollbar.set)

    h_scrollbar = ttk.Scrollbar(resultados_frame, orient="horizontal", command=matriz_resultados_canvas.xview)
    h_scrollbar.grid(row=1, column=0, sticky='ew')
    matriz_resultados_canvas.configure(xscrollcommand=h_scrollbar.set)

    global matriz_resultados_frame_inner
    matriz_resultados_frame_inner = tk.Frame(matriz_resultados_canvas, bg=background_color)
    matriz_resultados_canvas.create_window((0, 0), window=matriz_resultados_frame_inner, anchor='nw')

    def actualizar_scrollregion(event):
        matriz_resultados_canvas.configure(scrollregion=matriz_resultados_canvas.bbox("all"))

    matriz_resultados_frame_inner.bind("<Configure>", actualizar_scrollregion)
    root.grid_rowconfigure(11, weight=1)
    root.grid_columnconfigure(0, weight=1)
    resultados_frame.grid_rowconfigure(0, weight=1)
    resultados_frame.grid_columnconfigure(0, weight=1)

    # Cargar resultados y mostrar la tabla en la interfaz principal
    leer_resultados()
    actualizar_matriz_resultados()
    root.mainloop()

if __name__ == "__main__":
    crear_interfaz()


IndentationError: unexpected indent (651149739.py, line 493)

In [23]:
!pyinstaller --windowed Padl.py


2349 INFO: PyInstaller: 6.11.1, contrib hooks: 2024.10
2349 INFO: Python: 3.11.5 (conda)
2353 INFO: Platform: Windows-10-10.0.22621-SP0
2353 INFO: Python environment: C:\Users\imontero\AppData\Local\anaconda3
2354 INFO: wrote C:\Users\imontero\Padl\Padl.spec
2356 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Scripts\\pyinstaller.exe',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\imontero\\Padl']
5166 INFO: checking Analysis
5271 INFO: check

In [24]:
import shutil

# Crea padl_dist.zip a partir de la carpeta dist/Padl
shutil.make_archive("padl_dist", "zip", "dist/Padl")


'C:\\Users\\imontero\\Padl\\padl_dist.zip'

# DEFINITIVO

In [3]:
import itertools
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import csv
import json
from tkcalendar import DateEntry  # Importa DateEntry
from tkinter import PhotoImage
import sys
import os

# Librerías para gráficos y DataFrame
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
import numpy as np


import math
import json

# Archivo donde se guardarán los valores de Elo de los jugadores
archivo_elo = resource_path("ranking_elo.json")

# Cargar o inicializar el ranking Elo
def cargar_elo():
    if os.path.exists(archivo_elo):
        try:
            with open(archivo_elo, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            print("Error al leer el archivo de Elo:", e)
            return {jugador: 1500 for jugador in jugadores}
    else:
        return {jugador: 1500 for jugador in jugadores}

def guardar_elo(ranking_elo):
    with open(archivo_elo, "w", encoding="utf-8") as f:
        json.dump(ranking_elo, f, ensure_ascii=False, indent=4)

def calcular_probabilidad_elo(rating_A, rating_B):
    """
    Calcula la probabilidad de que el equipo A gane contra el equipo B usando la fórmula de Elo.
    """
    return 1 / (1 + math.pow(10, (rating_B - rating_A) / 400))

def recalcular_elo():
    """
    Recalcula el Elo desde cero considerando TODOS los partidos registrados.
    """
    global ranking_elo
    ranking_elo = {jugador: 1500 for jugador in jugadores}  # Resetear Elo de todos

    # Ordenar los partidos por fecha para procesarlos en orden cronológico
    partidos_ordenados = sorted(resultados, key=lambda x: x["fecha"])

    for partido in partidos_ordenados:
        actualizar_elo(partido, recalculando=True)  # Llamar a actualizar_elo con flag de recalculado

    guardar_elo(ranking_elo)  # Guardar los valores actualizados

def actualizar_elo(partido, recalculando=False):
    """
    Actualiza el ranking Elo de los jugadores después de un partido y retorna un diccionario 
    con la variación de Elo (positivo o negativo) que obtuvo cada jugador en ese partido.
    """
    equipo1, equipo2 = partido["partido"]
    ganador = partido["ganador_partido"]

    # Obtener los ratings actuales
    rating_1 = (ranking_elo[equipo1[0]] + ranking_elo[equipo1[1]]) / 2
    rating_2 = (ranking_elo[equipo2[0]] + ranking_elo[equipo2[1]]) / 2

    # Calcular probabilidades de victoria
    prob_1 = calcular_probabilidad_elo(rating_1, rating_2)
    prob_2 = calcular_probabilidad_elo(rating_2, rating_1)

    K = 32
    if set(ganador) == set(equipo1):
        resultado1, resultado2 = 1, 0
    else:
        resultado1, resultado2 = 0, 1

    # Guardamos los Elo previos para calcular la variación
    cambios_elo = {jugador: ranking_elo[jugador] for jugador in equipo1 + equipo2}

    # Actualizar Elo de cada jugador
    ranking_elo[equipo1[0]] = round(ranking_elo[equipo1[0]] + K * (resultado1 - prob_1))
    ranking_elo[equipo1[1]] = round(ranking_elo[equipo1[1]] + K * (resultado1 - prob_1))
    ranking_elo[equipo2[0]] = round(ranking_elo[equipo2[0]] + K * (resultado2 - prob_2))
    ranking_elo[equipo2[1]] = round(ranking_elo[equipo2[1]] + K * (resultado2 - prob_2))

    # Si no estamos recalculando, actualizamos el historial y guardamos el ranking
    if not recalculando:
        actualizar_historial_elo(partido["fecha"])
        guardar_elo(ranking_elo)

    # Calcular la variación (ganancia o pérdida) para cada jugador
    for jugador in cambios_elo:
        cambios_elo[jugador] = ranking_elo[jugador] - cambios_elo[jugador]

    return cambios_elo


def mostrar_ranking_elo():
    """
    Muestra el ranking Elo en una ventana con una tabla ordenada.
    """
    elo_window = tk.Toplevel()
    elo_window.title("Ranking Elo")

    ranking_ordenado = sorted(ranking_elo.items(), key=lambda x: x[1], reverse=True)

    frame = tk.Frame(elo_window)
    frame.pack(fill="both", expand=True)

    tree = ttk.Treeview(frame, columns=("Jugador", "Elo"), show="headings")
    tree.heading("Jugador", text="Jugador")
    tree.heading("Elo", text="Elo")
    tree.column("Jugador", anchor="center", width=150)
    tree.column("Elo", anchor="center", width=80)

    for jugador, elo in ranking_ordenado:
        tree.insert("", tk.END, values=(jugador, elo))

    tree.pack(expand=True, fill="both")

    ttk.Button(elo_window, text="Recalcular Elo", command=recalcular_elo).pack(pady=5)
    ttk.Button(elo_window, text="Cerrar", command=elo_window.destroy).pack(pady=5)




def mostrar_heatmap_sinergia():
    """
    Muestra un heatmap de sinergia entre jugadores.
    - El número en cada celda indica el total de partidos jugados juntos.
    - El color de la celda indica la cantidad de victorias logradas juntos (Blanco = pocas, Naranja = muchas).
    """
    import matplotlib
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    import numpy as np
    
    # Asegurar compatibilidad con Tkinter
    matplotlib.use("Agg")

    # 1. Construir diccionario de sinergia
    synergy = {}

    def _ordenar_par(p1, p2):
        """Ordena los nombres de los jugadores para evitar duplicados (p1, p2) == (p2, p1)"""
        return tuple(sorted([p1, p2]))

    for r in resultados:
        equipo1, equipo2 = r["partido"]
        ganador = set(r["ganador_partido"])  # Convertimos a conjunto para comparación rápida

        # Registrar equipo 1
        p1, p2 = equipo1
        par1 = _ordenar_par(p1, p2)
        synergy.setdefault(par1, {"jugados": 0, "ganados": 0})
        synergy[par1]["jugados"] += 1
        if set(equipo1) == ganador:
            synergy[par1]["ganados"] += 1

        # Registrar equipo 2
        p3, p4 = equipo2
        par2 = _ordenar_par(p3, p4)
        synergy.setdefault(par2, {"jugados": 0, "ganados": 0})
        synergy[par2]["jugados"] += 1
        if set(equipo2) == ganador:
            synergy[par2]["ganados"] += 1

    # 2. Crear matriz NxN con jugadores ordenados
    jugadores_ordenados = sorted(jugadores)
    n = len(jugadores_ordenados)

    # Matrices para datos
    matrix_jugados = np.zeros((n, n), dtype=int)  # Cantidad de partidos jugados juntos
    matrix_ganados = np.zeros((n, n), dtype=int)  # Cantidad de partidos ganados juntos

    # Llenar matrices
    for i in range(n):
        for j in range(n):
            if i == j:
                # Un jugador no hace pareja consigo mismo
                matrix_jugados[i, j] = 0
                matrix_ganados[i, j] = 0
            else:
                par = _ordenar_par(jugadores_ordenados[i], jugadores_ordenados[j])
                if par in synergy:
                    matrix_jugados[i, j] = synergy[par]["jugados"]
                    matrix_ganados[i, j] = synergy[par]["ganados"]

    # 3. Crear ventana Tkinter para mostrar el heatmap
    heatmap_window = tk.Toplevel()
    heatmap_window.title("Heatmap de Sinergia (Partidos Jugados y Ganados)")
    heatmap_window.geometry("900x600")

    fig, ax = plt.subplots(figsize=(7, 5))

    # 4. Crear el heatmap con los partidos ganados como colores (Blanco a Naranja)
    cax = ax.imshow(matrix_ganados, cmap="Oranges", interpolation="nearest")
    fig.colorbar(cax, ax=ax, label="Partidos Ganados Juntos")

    # Etiquetas en los ejes
    ax.set_xticks(range(n))
    ax.set_yticks(range(n))
    ax.set_xticklabels(jugadores_ordenados, rotation=45, ha="right")
    ax.set_yticklabels(jugadores_ordenados)
    ax.set_title("Sinergia: Partidos Jugados y Ganados Juntos")

    # 5. Añadir los números de partidos jugados dentro de las celdas
    for i in range(n):
        for j in range(n):
            if matrix_jugados[i, j] > 0:
                ax.text(j, i, str(matrix_jugados[i, j]), ha="center", va="center", color="black", fontsize=9, fontweight="bold")

    # Integrar en Tkinter
    canvas = FigureCanvasTkAgg(fig, master=heatmap_window)
    canvas.get_tk_widget().pack(expand=True, fill="both")
    canvas.draw()


def mostrar_grafico_elo():
    grafico_window = tk.Toplevel()
    grafico_window.title("Evolución del Ranking Elo")

    fig, ax = plt.subplots(figsize=(8, 5))

    if not historial_elo:
        messagebox.showwarning("Atención", "No hay datos de historial Elo disponibles.")
        return

    # Convertir las fechas de cada snapshot a datetime
    fechas = [pd.to_datetime(snapshot["fecha"], format="%Y-%m-%d") for snapshot in historial_elo]
    jugadores = list(ranking_elo.keys())

    for jugador in jugadores:
        valores_elo = [snapshot["ranking"].get(jugador, 1500) for snapshot in historial_elo]
        ax.plot(fechas, valores_elo, label=jugador, marker="o", linestyle="-")

    ax.set_title("Evolución del Elo a lo largo del tiempo")
    ax.set_xlabel("Fecha del Partido")
    ax.set_ylabel("Puntaje Elo")
    ax.legend(fontsize=8, loc="upper left", bbox_to_anchor=(1, 1))
    plt.xticks(rotation=45)
    plt.tight_layout()

    canvas = FigureCanvasTkAgg(fig, master=grafico_window)
    canvas.get_tk_widget().pack(expand=True, fill="both")
    canvas.draw()
    

def mostrar_grafico_jugadores():
    """
    Muestra un gráfico de dispersión donde cada punto (con un color único) representa un jugador.
    En el eje X se muestran los partidos jugados y en el eje Y se muestra, según la selección,
    el porcentaje de victorias o el total de victorias.
    """
    grafico_window = tk.Toplevel()
    grafico_window.title("Estadísticas de Jugadores")
    grafico_window.geometry("800x600")
    
    # Frame para controles (radiobuttons para seleccionar la métrica en Y)
    control_frame = tk.Frame(grafico_window)
    control_frame.pack(side="top", fill="x", pady=10)
    
    y_variable = tk.StringVar(value="porcentaje")
    
    rb_porcentaje = tk.Radiobutton(control_frame, text="Porcentaje de Victorias", 
                                   variable=y_variable, value="porcentaje")
    rb_porcentaje.pack(side="left", padx=10)
    rb_totales = tk.Radiobutton(control_frame, text="Victorias Totales", 
                                variable=y_variable, value="victorias")
    rb_totales.pack(side="left", padx=10)
    
    # Crear la figura para el gráfico
    fig, ax = plt.subplots(figsize=(8,6))
    canvas = FigureCanvasTkAgg(fig, master=grafico_window)
    canvas.get_tk_widget().pack(expand=True, fill="both")
    
    def update_graph(*args):
        # Seleccionar la métrica a graficar en el eje Y
        y_metric = y_variable.get()
        stats = calcular_estadisticas(resultados)
        players = list(stats.keys())
        
        # Eje X = partidos jugados
        x_values = [stats[player]["partidos_jugados"] for player in players]
        
        # Limpiar gráfico anterior
        ax.clear()
        
        if y_metric == "porcentaje":
            y_values = [stats[player]["porcentaje_victorias"] for player in players]
            ax.set_ylabel("Porcentaje de Victorias (%)")
        else:
            y_values = [stats[player]["victorias"] for player in players]
            ax.set_ylabel("Victorias Totales")
        
        ax.set_xlabel("Partidos Jugados")
        ax.set_title("Estadísticas de Jugadores")
        
        # Usar un colormap cualitativo para asignar colores a cada jugador
        colors = plt.cm.tab20.colors  # Tab20 ofrece hasta 20 colores distintos
        for i, player in enumerate(players):
            color = colors[i % len(colors)]
            ax.scatter(x_values[i], y_values[i], color=color, s=100, label=player)
            ax.annotate(player, 
                        (x_values[i], y_values[i]),
                        textcoords="offset points", xytext=(5,5), ha='left', fontsize=9)
        # Leyenda con el color y nombre
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
        canvas.draw()
    
    # Actualizar el gráfico cuando se cambie la selección
    y_variable.trace("w", update_graph)
    update_graph()


def mostrar_grafico_acumulado():
    """
    Crea un gráfico lineal donde cada jugador tiene una curva acumulativa 
    de partidos jugados a lo largo del tiempo (fecha).
    Cada vez que un jugador juega un partido, su línea sube en 1.
    """
    # 1. Construir un DataFrame con columnas ["fecha", "jugador", "valor"] = 1 por cada partido que jugó.
    df_rows = []
    for r in resultados:
        fecha_str = r["fecha"]
        # Los 4 jugadores en este partido
        jugadores_partido = r["partido"][0] + r["partido"][1]  # (equipo1, equipo2)
        for jug in jugadores_partido:
            df_rows.append({
                "fecha": fecha_str,
                "jugador": jug,
                "valor": 1  # indica que jugó un partido en esa fecha
            })
    
    if not df_rows:
        messagebox.showinfo("Info", "No hay datos para graficar.")
        return
    
    df = pd.DataFrame(df_rows)
    
    # 2. Convertir 'fecha' a datetime y descartar las filas inválidas
    df["fecha"] = pd.to_datetime(df["fecha"], format="%Y-%m-%d", errors="coerce")
    df.dropna(subset=["fecha"], inplace=True)
    
    # 3. Agrupar por (jugador, fecha) y contar cuántos partidos jugó ese día
    df_grouped = df.groupby(["jugador", "fecha"], as_index=False)["valor"].sum()
    # Ordenar por jugador y fecha
    df_grouped.sort_values(["jugador", "fecha"], inplace=True)
    
    # 4. Calcular el conteo acumulado (cumsum) por cada jugador
    df_grouped["cum_matches"] = df_grouped.groupby("jugador")["valor"].cumsum()
    
    # 5. Pivotar para tener cada jugador como columna y la fecha como índice
    df_pivot = df_grouped.pivot(index="fecha", columns="jugador", values="cum_matches")
    # Llenar huecos (si un jugador no jugó en cierta fecha)
    df_pivot = df_pivot.fillna(method="ffill").fillna(0)
    
    # 6. Crear ventana de Tkinter
    grafico_window = tk.Toplevel()
    grafico_window.title("Gráfico Acumulado de Partidos Jugados")
    grafico_window.geometry("1000x600")
    
    # Crear figura y eje
    fig, ax = plt.subplots(figsize=(9,5))
    
    # 7. Graficar cada jugador como una línea
    df_pivot.plot(ax=ax)
    
    # Ajustes del eje X para fechas
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Partidos Jugados (Acumulados)")
    ax.set_title("Evolución Acumulada de Partidos Jugados")
    fig.autofmt_xdate()  # Rotar etiquetas de fecha
    
    # Leyenda con colores y nombres
    ax.legend(title="Jugadores", bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Integrar el gráfico en la ventana
    canvas = FigureCanvasTkAgg(fig, master=grafico_window)
    canvas.get_tk_widget().pack(expand=True, fill="both")
    canvas.draw()


def resource_path(relative_path):
    """Obtener la ruta absoluta al recurso para ejecutables empaquetados."""
    try:
        # Si está empaquetado
        base_path = sys._MEIPASS
    except AttributeError:
        # Si no está empaquetado
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)

archivo_resultados = resource_path("resultados.csv")
archivo_jugadores = resource_path("jugadores.json")

# Funciones para persistir la lista de jugadores
def leer_jugadores():
    global jugadores
    if os.path.exists(archivo_jugadores):
        try:
            with open(archivo_jugadores, "r", encoding="utf-8") as f:
                jugadores = json.load(f)
        except Exception as e:
            print("Error al leer el archivo de jugadores:", e)
            jugadores = ["Ibai", "Xabi", "Ian", "Aitor", "Cifu", "David", "Igarki", "Aimar", "Erli", "Maria", "Dani", "AnderM", "Abad", "Sanchez"]
    else:
        # Lista inicial de jugadores
        jugadores = ["Ibai", "Xabi", "Ian", "Aitor", "Cifu", "David", "Igarki", "Aimar", "Erli", "Maria", "Dani", "AnderM", "Abad", "Sanchez"]

def guardar_jugadores():
    with open(archivo_jugadores, "w", encoding="utf-8") as f:
        json.dump(jugadores, f, ensure_ascii=False, indent=4)

# Cargar jugadores desde archivo al inicio
leer_jugadores()

def actualizar_datos_equipos():
    global parejas, equipos_str, equipo_str_a_pareja
    parejas = list(itertools.combinations(jugadores, 2))
    equipos_str = ["{} & {}".format(j1, j2) for (j1, j2) in parejas]
    equipo_str_a_pareja = dict(zip(equipos_str, parejas))

actualizar_datos_equipos()

resultados = []  # Aquí se guardan todos los partidos que se leen o se registran

def leer_resultados():
    if os.path.exists(archivo_resultados):
        with open(archivo_resultados, mode='r', newline='', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    equipo1_jugador1 = row.get("equipo1_jugador1", "").strip() or "Desconocido"
                    equipo1_jugador2 = row.get("equipo1_jugador2", "").strip() or "Desconocido"
                    equipo2_jugador1 = row.get("equipo2_jugador1", "").strip() or "Desconocido"
                    equipo2_jugador2 = row.get("equipo2_jugador2", "").strip() or "Desconocido"
                    ganador_primer_set_jugador1 = row.get("ganador_primer_set_jugador1", "").strip() or "Desconocido"
                    ganador_primer_set_jugador2 = row.get("ganador_primer_set_jugador2", "").strip() or "Desconocido"
                    ganador_partido_jugador1 = row.get("ganador_partido_jugador1", "").strip() or "Desconocido"
                    ganador_partido_jugador2 = row.get("ganador_partido_jugador2", "").strip() or "Desconocido"
                    puntuaciones = row.get("puntuaciones", "").strip().split(';') if row.get("puntuaciones") else []

                    resultado = {
                        "partido": (
                            (equipo1_jugador1, equipo1_jugador2),
                            (equipo2_jugador1, equipo2_jugador2)
                        ),
                        "ganador_primer_set": (ganador_primer_set_jugador1, ganador_primer_set_jugador2),
                        "ganador_partido": (ganador_partido_jugador1, ganador_partido_jugador2),
                        "mvp": row.get("mvp", "Desconocido").strip(),
                        "puntuaciones": puntuaciones,
                        "tie_breaks": int(row["tie_breaks"]) if row.get("tie_breaks") else 0,
                        "lugar": row.get("lugar", "Desconocido").strip(),
                        "fecha": row.get("fecha", "Desconocida").strip()
                    }
                    resultados.append(resultado)
                except Exception as e:
                    print(f"Error procesando fila: {row}, Error: {e}")

def guardar_resultado_csv(resultado):
    file_exists = os.path.exists(archivo_resultados)
    with open(archivo_resultados, mode='a', newline='', encoding='utf-8-sig') as file:
        fieldnames = [
            "equipo1_jugador1", "equipo1_jugador2",
            "equipo2_jugador1", "equipo2_jugador2",
            "ganador_primer_set_jugador1", "ganador_primer_set_jugador2",
            "ganador_partido_jugador1", "ganador_partido_jugador2",
            "mvp", "puntuaciones", "tie_breaks", "lugar", "fecha"
        ]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            "equipo1_jugador1": resultado["partido"][0][0],
            "equipo1_jugador2": resultado["partido"][0][1],
            "equipo2_jugador1": resultado["partido"][1][0],
            "equipo2_jugador2": resultado["partido"][1][1],
            "ganador_primer_set_jugador1": resultado["ganador_primer_set"][0],
            "ganador_primer_set_jugador2": resultado["ganador_primer_set"][1],
            "ganador_partido_jugador1": resultado["ganador_partido"][0],
            "ganador_partido_jugador2": resultado["ganador_partido"][1],
            "mvp": resultado["mvp"],
            "puntuaciones": ';'.join(resultado["puntuaciones"]),
            "tie_breaks": resultado["tie_breaks"],
            "lugar": resultado["lugar"],
            "fecha": resultado["fecha"]
        })

def calcular_estadisticas(resultados_filtrar):
    lugares = ["Ibaiondo", "Bakh", "Otro"]
    estadisticas = {}
    for jugador in jugadores:
        estadisticas[jugador] = {
            "partidos_jugados": 0,
            "victorias": 0,
            "mvp": 0,
            "sets_jugados": 0,
            "sets_ganados": 0,
            "tie_breaks": 0,
            "primer_set_ganado": 0,
            "games_ganados": 0,
            "games_perdidos": 0,
            "victorias_por_lugar": {lugar: 0 for lugar in lugares}
        }

    for resultado in resultados_filtrar:
        equipo1, equipo2 = resultado["partido"]
        ganador = resultado["ganador_partido"]
        mvp = resultado["mvp"]
        lugar = resultado["lugar"]
        sets_jugados = len(resultado["puntuaciones"])

        # Aseguramos que cualquier jugador inesperado aparezca en el diccionario
        for jugador in equipo1 + equipo2:
            if jugador not in estadisticas:
                estadisticas[jugador] = {
                    "partidos_jugados": 0,
                    "victorias": 0,
                    "mvp": 0,
                    "sets_jugados": 0,
                    "sets_ganados": 0,
                    "tie_breaks": 0,
                    "primer_set_ganado": 0,
                    "games_ganados": 0,
                    "games_perdidos": 0,
                    "victorias_por_lugar": {lugar: 0 for lugar in lugares}
                }

        # Partidos jugados y sets jugados
        for jugador in equipo1 + equipo2:
            estadisticas[jugador]["partidos_jugados"] += 1
            estadisticas[jugador]["sets_jugados"] += sets_jugados

        # Victoria del partido
        for jugador in ganador:
            estadisticas[jugador]["victorias"] += 1
            if lugar in estadisticas[jugador]["victorias_por_lugar"]:
                estadisticas[jugador]["victorias_por_lugar"][lugar] += 1

        # Ganador del primer set
        for jugador in resultado["ganador_primer_set"]:
            estadisticas[jugador]["primer_set_ganado"] += 1

        # Sets y games ganados/perdidos
        for set_result in resultado["puntuaciones"]:
            if '(' in set_result:
                score_part, _ = set_result.split('(')
                score1, score2 = map(int, score_part.split('-'))
                tie_breaks_in_set = 1
            else:
                score1, score2 = map(int, set_result.split('-'))
                tie_breaks_in_set = 0

            for jugador in equipo1 + equipo2:
                estadisticas[jugador]["tie_breaks"] += tie_breaks_in_set

            # Set ganado
            if score1 > score2:
                for jugador in equipo1:
                    estadisticas[jugador]["sets_ganados"] += 1
            else:
                for jugador in equipo2:
                    estadisticas[jugador]["sets_ganados"] += 1

            # Games
            for jugador in equipo1:
                estadisticas[jugador]["games_ganados"] += score1
                estadisticas[jugador]["games_perdidos"] += score2
            for jugador in equipo2:
                estadisticas[jugador]["games_ganados"] += score2
                estadisticas[jugador]["games_perdidos"] += score1

        # MVP
        if mvp in estadisticas:
            estadisticas[mvp]["mvp"] += 1

    # Calcular porcentajes y diferencias
    for jugador, stats in estadisticas.items():
        pj = stats["partidos_jugados"]
        if pj > 0:
            stats["porcentaje_victorias"] = stats["victorias"] / pj * 100
            stats["porcentaje_primer_set"] = stats["primer_set_ganado"] / pj * 100
        else:
            stats["porcentaje_victorias"] = 0
            stats["porcentaje_primer_set"] = 0
        stats["diferencia_games"] = stats["games_ganados"] - stats["games_perdidos"]

    return estadisticas

def generar_ranking(estadisticas):
    ranking = [(jugador, stats) for jugador, stats in estadisticas.items()]
    ranking.sort(key=lambda x: x[1]["porcentaje_victorias"], reverse=True)
    return ranking

def mostrar_ranking():
    estadisticas = calcular_estadisticas(resultados)

    años = sorted({r["fecha"][:4] for r in resultados})
    años.insert(0, "Todos")

    ranking_window = tk.Toplevel()
    ranking_window.title("Ranking de Jugadores")

    style = ttk.Style(ranking_window)
    style.theme_use('clam')
    style.configure('Treeview', background='#E3F2FD', foreground='black', rowheight=25, fieldbackground='#E3F2FD')
    style.configure('Treeview.Heading', background='#1E88E5', foreground='white', font=('Helvetica', 10, 'bold'))

    table_frame = tk.Frame(ranking_window)
    table_frame.pack(fill='both', expand=True)

    filtro_año_var = tk.StringVar(value="Todos")
    tk.Label(ranking_window, text="Filtrar por Año:", font=('Helvetica', 12)).pack(pady=5)
    filtro_año_combobox = ttk.Combobox(ranking_window, textvariable=filtro_año_var, values=años, state='readonly')
    filtro_año_combobox.pack(pady=5)

    tree_scroll_y = ttk.Scrollbar(table_frame, orient='vertical')
    tree_scroll_y.pack(side='right', fill='y')

    tree_scroll_x = ttk.Scrollbar(table_frame, orient='horizontal')
    tree_scroll_x.pack(side='bottom', fill='x')

    columns = (
        "Posición", "Jugador", "Porc. Victorias", "Porc. 1er Set", "Sets Jugados",
        "Victorias", "Partidos Jugados", "Tie Breaks", "MVPs", "Games Ganados",
        "Games Perdidos", "Dif. Games", "Victorias en Ibaiondo", "Victorias en Bakh", "Victorias en Otro"
    )
    tree = ttk.Treeview(table_frame, columns=columns, show='headings',
                        yscrollcommand=tree_scroll_y.set, xscrollcommand=tree_scroll_x.set)
    tree_scroll_y.config(command=tree.yview)
    tree_scroll_x.config(command=tree.xview)

    column_widths = {
        "Posición": 60, "Jugador": 100, "Porc. Victorias": 100, "Porc. 1er Set": 100,
        "Sets Jugados": 90, "Victorias": 70, "Partidos Jugados": 110, "Tie Breaks": 80,
        "MVPs": 60, "Games Ganados": 100, "Games Perdidos": 100, "Dif. Games": 80,
        "Victorias en Ibaiondo": 130, "Victorias en Bakh": 120, "Victorias en Otro": 110
    }

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, anchor='center', width=column_widths.get(col, 100), stretch=False)

    tree.pack(expand=True, fill='both')

    def actualizar_ranking():
        año_seleccionado = filtro_año_var.get()
        if año_seleccionado == "Todos":
            resultados_filtrados = resultados
        else:
            resultados_filtrados = [r for r in resultados if r["fecha"].startswith(año_seleccionado)]

        est_filtradas = calcular_estadisticas(resultados_filtrados)
        ranking_filtrado = generar_ranking(est_filtradas)

        for item in tree.get_children():
            tree.delete(item)

        posicion = 1
        for jugador, stats in ranking_filtrado:
            valores = [
                posicion,
                jugador,
                f"{stats['porcentaje_victorias']:.2f}%",
                f"{stats['porcentaje_primer_set']:.2f}%",
                stats['sets_jugados'],
                stats['victorias'],
                stats['partidos_jugados'],
                stats['tie_breaks'],
                stats['mvp'],
                stats['games_ganados'],
                stats['games_perdidos'],
                stats['diferencia_games'],
                stats['victorias_por_lugar'].get("Ibaiondo", 0),
                stats['victorias_por_lugar'].get("Bakh", 0),
                stats['victorias_por_lugar'].get("Otro", 0)
            ]
            tree.insert('', tk.END, values=valores)
            posicion += 1

    filtro_año_combobox.bind("<<ComboboxSelected>>", lambda e: actualizar_ranking())
    actualizar_ranking()

    ttk.Button(ranking_window, text="Cerrar", command=ranking_window.destroy).pack(pady=10)

    # Ordenar columnas al hacer clic
    tree_sort_column = {'column': None, 'descending': False}
    def sort_treeview(treeview, col):
        data = []
        for child in treeview.get_children():
            item = treeview.item(child)
            values = list(item['values'])
            data.append((values, child))

        col_indices = {columns[i]: i for i in range(len(columns))}
        col_index = col_indices[col]

        if tree_sort_column['column'] == col:
            descending = not tree_sort_column['descending']
        else:
            descending = False
        tree_sort_column['column'] = col
        tree_sort_column['descending'] = descending

        def convert(value):
            try:
                if isinstance(value, str) and value.endswith('%'):
                    return float(value.rstrip('%'))
                return float(value)
            except ValueError:
                return value.lower() if isinstance(value, str) else value

        data.sort(key=lambda x: convert(x[0][col_index]), reverse=descending)

        for index, (values, item) in enumerate(data):
            values[0] = index + 1
            treeview.item(item, values=values)
            treeview.move(item, '', index)

    for col in columns:
        tree.heading(col, text=col, command=lambda _col=col: sort_treeview(tree, _col))

def mostrar_partidos():
    """Abre una ventana con todos los partidos y permite filtrar por un jugador."""
    partidos_window = tk.Toplevel()
    partidos_window.title("Lista de Partidos")

    style = ttk.Style(partidos_window)
    style.theme_use('clam')
    style.configure('Treeview', background='#E3F2FD', foreground='black', rowheight=25, fieldbackground='#E3F2FD')
    style.configure('Treeview.Heading', background='#1E88E5', foreground='white', font=('Helvetica', 10, 'bold'))

    filtro_frame = tk.Frame(partidos_window)
    filtro_frame.pack(pady=5)

    tk.Label(filtro_frame, text="Filtrar por Jugador:", font=('Helvetica', 12)).grid(row=0, column=0, padx=5)
    jugador_filtro_var = tk.StringVar(value="Todos")
    lista_jugadores_filtro = ["Todos"] + jugadores
    jugador_filtro_combobox = ttk.Combobox(filtro_frame, textvariable=jugador_filtro_var,
                                           values=lista_jugadores_filtro, state='readonly')
    jugador_filtro_combobox.grid(row=0, column=1, padx=5)

    table_frame = tk.Frame(partidos_window)
    table_frame.pack(fill='both', expand=True)

    scroll_y = ttk.Scrollbar(table_frame, orient='vertical')
    scroll_y.pack(side='right', fill='y')

    scroll_x = ttk.Scrollbar(table_frame, orient='horizontal')
    scroll_x.pack(side='bottom', fill='x')

    columnas = ["Fecha", "Equipo 1", "Equipo 2", "Puntuaciones", "Ganador", "MVP", "Tie-breaks", "Lugar"]
    tree_partidos = ttk.Treeview(table_frame, columns=columnas, show='headings',
                                 yscrollcommand=scroll_y.set, xscrollcommand=scroll_x.set)
    scroll_y.config(command=tree_partidos.yview)
    scroll_x.config(command=tree_partidos.xview)

    for col in columnas:
        tree_partidos.heading(col, text=col)
        tree_partidos.column(col, anchor='center', width=120, stretch=False)

    tree_partidos.pack(expand=True, fill='both')

    def actualizar_partidos():
        tree_partidos.delete(*tree_partidos.get_children())
        jugador_seleccionado = jugador_filtro_var.get()

        for r in resultados:
            fecha = r["fecha"]
            eq1_str = " & ".join(r["partido"][0])
            eq2_str = " & ".join(r["partido"][1])
            puntuaciones = "; ".join(r["puntuaciones"]) if r["puntuaciones"] else "N/A"
            ganador_partido = " & ".join(r["ganador_partido"])
            mvp = r["mvp"]
            tie_breaks = r["tie_breaks"]
            lugar = r["lugar"]

            if jugador_seleccionado == "Todos":
                mostrar = True
            else:
                jugadores_partido = r["partido"][0] + r["partido"][1]
                mostrar = (jugador_seleccionado in jugadores_partido)

            if mostrar:
                tree_partidos.insert('', tk.END, values=(fecha, eq1_str, eq2_str, puntuaciones, ganador_partido, mvp, tie_breaks, lugar))

    boton_filtrar = tk.Button(filtro_frame, text="Filtrar", command=actualizar_partidos, bg='#1E88E5', fg='white')
    boton_filtrar.grid(row=0, column=2, padx=5)

    ttk.Button(partidos_window, text="Cerrar", command=partidos_window.destroy).pack(pady=5)
    actualizar_partidos()

def crear_interfaz():
    root = tk.Tk()
    root.title("Registrar Resultado de Partido")
    
    # Configuración de estilo y colores (apariencia moderna)
    style = ttk.Style(root)
    style.theme_use('clam')
    primary_color = '#1E88E5'
    background_color = '#E3F2FD'
    root.configure(bg=background_color)
    style.configure('TButton', font=('Segoe UI', 10), padding=5)
    style.configure('TLabel', font=('Segoe UI', 10))
    style.configure('TCombobox', font=('Segoe UI', 10))
    
    # --- MENÚ DE NAVEGACIÓN ---
    menu_bar = tk.Menu(root)
    navegacion_menu = tk.Menu(menu_bar, tearoff=0)
    navegacion_menu.add_command(label="Gráfico Jugadores", command=mostrar_grafico_jugadores)
    navegacion_menu.add_command(label="Gráfico Acumulado", command=mostrar_grafico_acumulado)
    navegacion_menu.add_command(label="Heatmap de Sinergia", command=mostrar_heatmap_sinergia)
    navegacion_menu.add_command(label="Ranking Elo", command=mostrar_ranking_elo)

    #navegacion_menu.add_command(label="Registro de Partido", command=lambda: root.focus())
    #navegacion_menu.add_command(label="Mostrar Ranking", command=mostrar_ranking)
    #navegacion_menu.add_command(label="Mostrar Partidos", command=mostrar_partidos)
    #navegacion_menu.add_command(label="Gestión de Jugadores", command=lambda: gestionar_jugadores())
    menu_bar.add_cascade(label="Navegación", menu=navegacion_menu)
    root.config(menu=menu_bar)
    # --- FIN MENÚ ---
    
    set_resultados = {}
    tie_break_vars = {}
    tie_break_scores = {}

    def toggle_tiebreak_entry(set_num):
        if tie_break_vars[set_num].get():
            tie_break_scores[set_num].config(state='normal')
        else:
            tie_break_scores[set_num].delete(0, tk.END)
            tie_break_scores[set_num].config(state='disabled')

    def registrar():
        equipo1_str = equipo1_var.get()
        equipo2_str = equipo2_var.get()
        fecha = fecha_var.get_date()

        if not equipo1_str or not equipo2_str:
            messagebox.showerror("Error", "Debes seleccionar ambos equipos.")
            return
        if equipo1_str == equipo2_str:
            messagebox.showerror("Error", "Los equipos no pueden ser iguales.")
            return

        equipo1 = equipo_str_a_pareja.get(equipo1_str)
        equipo2 = equipo_str_a_pareja.get(equipo2_str)
        if not equipo1 or not equipo2:
            messagebox.showerror("Error", "Equipos no válidos.")
            return

        ganador_primer_set_str = ganador_primer_set_var.get()
        ganador_partido_str = ganador_partido_var.get()
        if not ganador_primer_set_str or not ganador_partido_str:
            messagebox.showerror("Error", "Debes seleccionar los equipos ganadores.")
            return

        ganador_primer_set = equipo1 if ganador_primer_set_str == equipo1_str else equipo2
        ganador_partido = equipo1 if ganador_partido_str == equipo1_str else equipo2

        mvp = mvp_jugador.get()
        if not mvp:
            messagebox.showerror("Error", "Debes seleccionar el MVP.")
            return

        puntuaciones = []
        tie_breaks = 0
        for set_num in range(1, 4):
            set_resultado = set_resultados[set_num].get()
            tie_break_var = tie_break_vars[set_num]
            tie_break_score = tie_break_scores[set_num].get()

            if set_resultado:
                try:
                    score1, score2 = map(int, set_resultado.split('-'))
                except ValueError:
                    messagebox.showerror("Error", f"El resultado del Set {set_num} debe ser 'n-n'.")
                    return
                if tie_break_var.get():
                    if not tie_break_score:
                        messagebox.showerror("Error", f"Debes ingresar la puntuación del Tie-break para el Set {set_num}.")
                        return
                    puntuacion = f"{set_resultado}({tie_break_score})"
                    tie_breaks += 1
                else:
                    puntuacion = set_resultado
                puntuaciones.append(puntuacion)

        lugar = lugar_var.get()
        resultado = {
            "partido": (equipo1, equipo2),
            "ganador_primer_set": ganador_primer_set,
            "ganador_partido": ganador_partido,
            "mvp": mvp,
            "puntuaciones": puntuaciones,
            "tie_breaks": tie_breaks,
            "lugar": lugar,
            "fecha": fecha.strftime('%Y-%m-%d')
        }
        resultados.append(resultado)
        guardar_resultado_csv(resultado)
        messagebox.showinfo("Éxito", "Resultado registrado exitosamente")

        # Limpiar campos
        equipo1_var.set("")
        equipo2_var.set("")
        ganador_primer_set_var.set("")
        ganador_partido_var.set("")
        mvp_jugador.set("")
        for set_num in range(1, 4):
            set_resultados[set_num].delete(0, tk.END)
            tie_break_vars[set_num].set(False)
            tie_break_scores[set_num].delete(0, tk.END)
            tie_break_scores[set_num].config(state='disabled')
        lugar_var.set("Ibaiondo")

        ganador_primer_set_combobox['values'] = []
        ganador_partido_combobox['values'] = []

        actualizar_matriz_resultados()

    def actualizar_ganadores(*args):
        equipo1_str = equipo1_var.get()
        equipo2_str = equipo2_var.get()
        if equipo1_str and equipo2_str:
            opciones_ganadores = [equipo1_str, equipo2_str]
            ganador_primer_set_combobox['values'] = opciones_ganadores
            ganador_partido_combobox['values'] = opciones_ganadores
        else:
            ganador_primer_set_combobox['values'] = []
            ganador_partido_combobox['values'] = []

    def actualizar_matriz_resultados():
        for widget in matriz_resultados_frame_inner.winfo_children():
            widget.destroy()

        encabezados = ["Equipo 1", "Equipo 2", "Ganador 1er Set", "Ganador Partido", "MVP", "Puntuaciones", "Tie Breaks", "Lugar", "Fecha"]
        for i, resultado in enumerate(resultados, start=1):
            try:
                eq1 = " & ".join(resultado["partido"][0])
                eq2 = " & ".join(resultado["partido"][1])
                g1set = " & ".join(resultado["ganador_primer_set"])
                gpart = " & ".join(resultado["ganador_partido"])
                mvp = resultado["mvp"]
                punts = "; ".join(resultado["puntuaciones"])
                tb = resultado["tie_breaks"]
                lugar = resultado["lugar"]
                fecha = resultado["fecha"]

                datos = [eq1, eq2, g1set, gpart, mvp, punts, tb, lugar, fecha]
                for idx, dato in enumerate(datos):
                    label = tk.Label(matriz_resultados_frame_inner, text=dato, bg=background_color)
                    label.grid(row=i, column=idx, padx=5, pady=5)
                    label.configure(borderwidth=1, relief="solid")
            except Exception as e:
                print(f"Error mostrando resultado: {resultado}, Error: {e}")

        matriz_resultados_canvas.configure(scrollregion=matriz_resultados_canvas.bbox("all"))

    def actualizar_combos():
        actualizar_datos_equipos()
        equipo1_combobox['values'] = equipos_str
        equipo2_combobox['values'] = equipos_str
        mvp_jugador['values'] = jugadores

    def gestionar_jugadores():
        gestion_window = tk.Toplevel(root)
        gestion_window.title("Gestión de Jugadores")
        gestion_window.configure(bg=background_color)
        
        listbox = tk.Listbox(gestion_window, font=('Helvetica', 10))
        listbox.pack(side='left', fill='both', expand=True, padx=5, pady=5)
        scrollbar = ttk.Scrollbar(gestion_window, orient="vertical", command=listbox.yview)
        scrollbar.pack(side='left', fill='y')
        listbox.config(yscrollcommand=scrollbar.set)
        
        def actualizar_listbox():
            listbox.delete(0, tk.END)
            for p in jugadores:
                listbox.insert(tk.END, p)
        actualizar_listbox()
        
        btn_frame = tk.Frame(gestion_window, bg=background_color)
        btn_frame.pack(side='bottom', fill='x', padx=5, pady=5)
        
        def agregar_jugador():
            new_name = simpledialog.askstring("Agregar Jugador", "Nombre del nuevo jugador:")
            if new_name:
                new_name = new_name.strip()
                if new_name in jugadores:
                    messagebox.showerror("Error", "El jugador ya existe.")
                else:
                    jugadores.append(new_name)
                    actualizar_listbox()
                    actualizar_combos()
                    guardar_jugadores()  # Guardamos los cambios
       
        def editar_jugador():
            selected = listbox.curselection()
            if not selected:
                messagebox.showerror("Error", "Selecciona un jugador para editar.")
                return
            index = selected[0]
            current_name = jugadores[index]
            new_name = simpledialog.askstring("Editar Jugador", "Nuevo nombre para el jugador:", initialvalue=current_name)
            if new_name:
                new_name = new_name.strip()
                if new_name in jugadores:
                    messagebox.showerror("Error", "El nombre ya existe.")
                else:
                    jugadores[index] = new_name
                    actualizar_listbox()
                    actualizar_combos()
                    guardar_jugadores()  # Guardamos los cambios
        
        def eliminar_jugador():
            selected = listbox.curselection()
            if not selected:
                messagebox.showerror("Error", "Selecciona un jugador para eliminar.")
                return
            index = selected[0]
            jugador_eliminar = jugadores[index]
            if messagebox.askyesno("Confirmar", f"¿Estás seguro de eliminar a {jugador_eliminar}?"):
                del jugadores[index]
                actualizar_listbox()
                actualizar_combos()
                guardar_jugadores()  # Guardamos los cambios
        
        btn_agregar = tk.Button(btn_frame, text="Agregar", command=agregar_jugador, bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
        btn_agregar.pack(side='left', padx=5, pady=5)
        btn_editar = tk.Button(btn_frame, text="Editar", command=editar_jugador, bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
        btn_editar.pack(side='left', padx=5, pady=5)
        btn_eliminar = tk.Button(btn_frame, text="Eliminar", command=eliminar_jugador, bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
        btn_eliminar.pack(side='left', padx=5, pady=5)
        btn_cerrar = tk.Button(btn_frame, text="Cerrar", command=gestion_window.destroy, bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
        btn_cerrar.pack(side='right', padx=5, pady=5)

    # CREACIÓN DE WIDGETS DEL FORMULARIO
    tk.Label(root, text="Fecha del Partido:", font=('Helvetica', 12), bg=background_color).grid(row=0, column=0, sticky='e')
    fecha_var = DateEntry(root, width=12, background='darkblue', foreground='white', borderwidth=2, date_pattern='y-mm-dd')
    fecha_var.grid(row=0, column=1, pady=5, padx=5)

    tk.Label(root, text="Equipo 1:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=1, column=0, sticky='e')
    equipo1_var = tk.StringVar()
    equipo1_combobox = ttk.Combobox(root, textvariable=equipo1_var, values=equipos_str)
    equipo1_combobox.grid(row=1, column=1, pady=5, padx=5)

    tk.Label(root, text="Equipo 2:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=2, column=0, sticky='e')
    equipo2_var = tk.StringVar()
    equipo2_combobox = ttk.Combobox(root, textvariable=equipo2_var, values=equipos_str)
    equipo2_combobox.grid(row=2, column=1, pady=5, padx=5)

    equipo1_var.trace('w', actualizar_ganadores)
    equipo2_var.trace('w', actualizar_ganadores)

    tk.Label(root, text="Ganador 1er Set:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=3, column=0, sticky='e')
    ganador_primer_set_var = tk.StringVar()
    ganador_primer_set_combobox = ttk.Combobox(root, textvariable=ganador_primer_set_var)
    ganador_primer_set_combobox.grid(row=3, column=1, pady=5, padx=5)

    tk.Label(root, text="Ganador Partido:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=4, column=0, sticky='e')
    ganador_partido_var = tk.StringVar()
    ganador_partido_combobox = ttk.Combobox(root, textvariable=ganador_partido_var)
    ganador_partido_combobox.grid(row=4, column=1, pady=5, padx=5)

    tk.Label(root, text="MVP:", font=('Helvetica', 12, 'bold'), bg=background_color).grid(row=5, column=0, sticky='e')
    mvp_jugador = ttk.Combobox(root, values=jugadores)
    mvp_jugador.grid(row=5, column=1, pady=5, padx=5)

    set_resultados = {}
    tie_break_vars = {}
    tie_break_scores = {}

    for set_num in range(1, 4):
        tk.Label(root, text=f"Set {set_num} (ej: 6-4):", font=('Helvetica', 12), bg=background_color).grid(row=5 + set_num, column=0, sticky='e')
        set_resultados[set_num] = ttk.Entry(root)
        set_resultados[set_num].grid(row=5 + set_num, column=1, pady=5, padx=5)

        tie_break_vars[set_num] = tk.BooleanVar()
        tie_break_cb = tk.Checkbutton(root, text="Tie-break", variable=tie_break_vars[set_num], bg=background_color,
                                      command=lambda num=set_num: toggle_tiebreak_entry(num))
        tie_break_cb.grid(row=5 + set_num, column=2, padx=5)

        tk.Label(root, text=f"Puntuación Tie-break Set {set_num}:", font=('Helvetica', 12), bg=background_color).grid(row=5 + set_num, column=3, sticky='e')
        tie_break_scores[set_num] = ttk.Entry(root, state='disabled')
        tie_break_scores[set_num].grid(row=5 + set_num, column=4, pady=5, padx=5)

    tk.Label(root, text="Lugar del Partido:", font=('Helvetica', 12), bg=background_color).grid(row=9, column=0, sticky='e')
    lugar_var = tk.StringVar(value="Ibaiondo")
    lugar_options = ["Ibaiondo", "Bakh", "Otro"]
    lugar_menu = ttk.Combobox(root, textvariable=lugar_var, values=lugar_options)
    lugar_menu.grid(row=9, column=1, pady=5, padx=5)

    botones_frame = tk.Frame(root, bg=background_color)
    botones_frame.grid(row=10, columnspan=5, pady=10)

    registrar_button = tk.Button(botones_frame, text="Registrar Resultado", command=registrar,
                                 bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
    registrar_button.grid(row=0, column=0, padx=5)
    ranking_button = tk.Button(botones_frame, text="Mostrar Ranking", command=mostrar_ranking,
                               bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
    ranking_button.grid(row=0, column=1, padx=5)
    partidos_button = tk.Button(botones_frame, text="Mostrar Partidos", command=mostrar_partidos,
                                bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
    partidos_button.grid(row=0, column=2, padx=5)
    gestion_button = tk.Button(botones_frame, text="Gestión de Jugadores", command=gestionar_jugadores,
                               bg=primary_color, fg='white', font=('Helvetica', 10, 'bold'))
    gestion_button.grid(row=0, column=3, padx=5)

    resultados_frame = tk.Frame(root, bg=background_color)
    resultados_frame.grid(row=11, columnspan=5, pady=10, sticky='nsew')

    global matriz_resultados_canvas
    matriz_resultados_canvas = tk.Canvas(resultados_frame, bg=background_color)
    matriz_resultados_canvas.grid(row=0, column=0, sticky='nsew')

    scrollbar = ttk.Scrollbar(resultados_frame, orient="vertical", command=matriz_resultados_canvas.yview)
    scrollbar.grid(row=0, column=1, sticky='ns')
    matriz_resultados_canvas.configure(yscrollcommand=scrollbar.set)

    h_scrollbar = ttk.Scrollbar(resultados_frame, orient="horizontal", command=matriz_resultados_canvas.xview)
    h_scrollbar.grid(row=1, column=0, sticky='ew')
    matriz_resultados_canvas.configure(xscrollcommand=h_scrollbar.set)

    global matriz_resultados_frame_inner
    matriz_resultados_frame_inner = tk.Frame(matriz_resultados_canvas, bg=background_color)
    matriz_resultados_canvas.create_window((0, 0), window=matriz_resultados_frame_inner, anchor='nw')

    def actualizar_scrollregion(event):
        matriz_resultados_canvas.configure(scrollregion=matriz_resultados_canvas.bbox("all"))
    matriz_resultados_frame_inner.bind("<Configure>", actualizar_scrollregion)
    
    root.grid_rowconfigure(11, weight=1)
    root.grid_columnconfigure(0, weight=1)
    resultados_frame.grid_rowconfigure(0, weight=1)
    resultados_frame.grid_columnconfigure(0, weight=1)

    # Inicializar las listas y widgets dependientes
    actualizar_combos()
    leer_resultados()
    actualizar_matriz_resultados()
    root.mainloop()

if __name__ == "__main__":
    crear_interfaz()


NameError: name 'resource_path' is not defined

# Mejoras de Elo

In [5]:
import itertools
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import csv
import json
from tkcalendar import DateEntry  # pip install tkcalendar
import sys, os, math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime

# -------------------------------------------------------------------
# 1. Función resource_path (si empaquetas con PyInstaller)
# -------------------------------------------------------------------
def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except AttributeError:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)

# -------------------------------------------------------------------
# 2. Función para determinar la Season según la fecha
#    - Season 0: antes de 1/1/2025
#    - A partir de 1/1/2025, Season N cada 6 meses
# -------------------------------------------------------------------
def obtener_season(fecha_str):
    """
    Recibe una fecha en formato 'YYYY-MM-DD' y retorna:
      - "Season 0" si la fecha es anterior al 1/1/2025.
      - A partir del 1/1/2025, "Season N" donde N incrementa cada 6 meses.
    """
    try:
        fecha = datetime.strptime(fecha_str, '%Y-%m-%d')
    except Exception:
        return "Unknown"

    limite = datetime(2025, 1, 1)
    if fecha < limite:
        return "Season 0"
    else:
        # Calcular la "semestre" desde 1/1/2025
        # Cada año tiene 2 seasons: 1er semestre y 2do semestre
        # Season 1 = 1er semestre 2025 (ene-jun), Season 2 = 2do semestre 2025 (jul-dic), etc.
        # Podrías ajustar la lógica si lo deseas.
        year_diff = fecha.year - 2025
        # 0 si es primer semestre, 1 si es segundo
        if fecha.month <= 6:
            sem = 0
        else:
            sem = 1
        # Season = 1 + (2 * year_diff) + sem
        # Ej: 1er semestre 2025 => year_diff=0, sem=0 => Season=1
        #     2do semestre 2025 => year_diff=0, sem=1 => Season=2
        season_num = 1 + (2 * year_diff) + sem
        return f"Season {season_num}"

# -------------------------------------------------------------------
# 3. Estructuras principales
# -------------------------------------------------------------------
jugadores = []
resultados = []  # lista de partidos leídos desde CSV
parejas = []
equipos_str = []
equipo_str_a_pareja = {}

# Aquí almacenaremos el Elo final de cada Season:
#    ranking_elo_por_season = {
#        "Season 0": {"Ibai": EloFinal, "Xabi": EloFinal, ...},
#        "Season 1": {...},
#        ...
#    }
ranking_elo_por_season = {}

# Aquí guardamos la variación de Elo por partido:
#   elo_changes_por_partido = {
#       index_en_resultados: { "Ibai": +10, "Xabi": -10, ... }
#   }
elo_changes_por_partido = {}

# -------------------------------------------------------------------
# 4. Leer y guardar jugadores
# -------------------------------------------------------------------
archivo_jugadores = resource_path("jugadores.json")
def leer_jugadores():
    global jugadores
    if os.path.exists(archivo_jugadores):
        try:
            with open(archivo_jugadores, "r", encoding="utf-8") as f:
                jugadores = json.load(f)
        except Exception as e:
            print("Error al leer el archivo de jugadores:", e)
            jugadores = []
    else:
        # Lista inicial (ajústala si quieres)
        jugadores = ["Ibai", "Xabi", "Ian", "Aitor", "Cifu", "David",
                     "Igarki", "Aimar", "Erli", "Maria", "Dani", "AnderM",
                     "Abad", "Sanchez"]

def guardar_jugadores():
    with open(archivo_jugadores, "w", encoding="utf-8") as f:
        json.dump(jugadores, f, ensure_ascii=False, indent=4)

# -------------------------------------------------------------------
# 5. Leer resultados desde CSV
# -------------------------------------------------------------------
archivo_resultados = resource_path("resultados.csv")

def leer_resultados():
    global resultados
    resultados.clear()
    if os.path.exists(archivo_resultados):
        with open(archivo_resultados, mode='r', newline='', encoding='utf-8-sig') as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    eq1j1 = row.get("equipo1_jugador1", "").strip()
                    eq1j2 = row.get("equipo1_jugador2", "").strip()
                    eq2j1 = row.get("equipo2_jugador1", "").strip()
                    eq2j2 = row.get("equipo2_jugador2", "").strip()
                    g1s1 = row.get("ganador_primer_set_jugador1", "").strip()
                    g1s2 = row.get("ganador_primer_set_jugador2", "").strip()
                    gpart1 = row.get("ganador_partido_jugador1", "").strip()
                    gpart2 = row.get("ganador_partido_jugador2", "").strip()
                    puntuaciones = row.get("puntuaciones", "").split(';') if row.get("puntuaciones") else []
                    fecha_str = row.get("fecha", "").strip()
                    season = row.get("season")  # puede venir en CSV o no
                    if not season:
                        season = obtener_season(fecha_str)

                    resultado = {
                        "partido": ((eq1j1, eq1j2), (eq2j1, eq2j2)),
                        "ganador_primer_set": (g1s1, g1s2),
                        "ganador_partido": (gpart1, gpart2),
                        "mvp": row.get("mvp", "").strip(),
                        "puntuaciones": puntuaciones,
                        "tie_breaks": int(row["tie_breaks"]) if row.get("tie_breaks") else 0,
                        "lugar": row.get("lugar", "").strip(),
                        "fecha": fecha_str,
                        "season": season
                    }
                    resultados.append(resultado)
                except Exception as e:
                    print(f"Error procesando fila: {row}, Error: {e}")

# -------------------------------------------------------------------
# 6. Guardar resultado en CSV (cuando se registra un partido nuevo)
# -------------------------------------------------------------------
def guardar_resultado_csv(resultado):
    file_exists = os.path.exists(archivo_resultados)
    fieldnames = [
        "equipo1_jugador1", "equipo1_jugador2",
        "equipo2_jugador1", "equipo2_jugador2",
        "ganador_primer_set_jugador1", "ganador_primer_set_jugador2",
        "ganador_partido_jugador1", "ganador_partido_jugador2",
        "mvp", "puntuaciones", "tie_breaks", "lugar", "fecha", "season"
    ]
    with open(archivo_resultados, mode='a', newline='', encoding='utf-8-sig') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            "equipo1_jugador1": resultado["partido"][0][0],
            "equipo1_jugador2": resultado["partido"][0][1],
            "equipo2_jugador1": resultado["partido"][1][0],
            "equipo2_jugador2": resultado["partido"][1][1],
            "ganador_primer_set_jugador1": resultado["ganador_primer_set"][0],
            "ganador_primer_set_jugador2": resultado["ganador_primer_set"][1],
            "ganador_partido_jugador1": resultado["ganador_partido"][0],
            "ganador_partido_jugador2": resultado["ganador_partido"][1],
            "mvp": resultado["mvp"],
            "puntuaciones": ';'.join(resultado["puntuaciones"]),
            "tie_breaks": resultado["tie_breaks"],
            "lugar": resultado["lugar"],
            "fecha": resultado["fecha"],
            "season": resultado["season"]
        })

# -------------------------------------------------------------------
# 7. Funciones para recalcular Elo por Season
# -------------------------------------------------------------------
def actualizar_datos_equipos():
    global parejas, equipos_str, equipo_str_a_pareja
    parejas = list(itertools.combinations(jugadores, 2))
    equipos_str = ["{} & {}".format(j1, j2) for (j1, j2) in parejas]
    equipo_str_a_pareja = dict(zip(equipos_str, parejas))

def calcular_probabilidad_elo(rA, rB):
    return 1 / (1 + math.pow(10, (rB - rA) / 400))

def actualizar_elo_sin_guardar(ranking_local, partido):
    """
    Dado un dict 'ranking_local' con Elo actual de cada jugador,
    y un partido (diccionario con 'partido', 'ganador_partido', etc.),
    actualiza los Elo en ranking_local y retorna un dict con la variación.
    """
    equipo1, equipo2 = partido["partido"]
    ganador = partido["ganador_partido"]

    # Elo medio de cada equipo
    rating_1 = (ranking_local[equipo1[0]] + ranking_local[equipo1[1]]) / 2
    rating_2 = (ranking_local[equipo2[0]] + ranking_local[equipo2[1]]) / 2

    prob_1 = calcular_probabilidad_elo(rating_1, rating_2)
    prob_2 = calcular_probabilidad_elo(rating_2, rating_1)

    K = 32
    if set(ganador) == set(equipo1):
        res1, res2 = 1, 0
    else:
        res1, res2 = 0, 1

    cambios = {}
    # Guardamos Elo previo de cada jugador para calcular la variación
    for j in equipo1 + equipo2:
        cambios[j] = ranking_local[j]

    # Actualizar
    ranking_local[equipo1[0]] = round(ranking_local[equipo1[0]] + K*(res1 - prob_1))
    ranking_local[equipo1[1]] = round(ranking_local[equipo1[1]] + K*(res1 - prob_1))
    ranking_local[equipo2[0]] = round(ranking_local[equipo2[0]] + K*(res2 - prob_2))
    ranking_local[equipo2[1]] = round(ranking_local[equipo2[1]] + K*(res2 - prob_2))

    # Calcular variación real
    for j in equipo1 + equipo2:
        cambios[j] = ranking_local[j] - cambios[j]
    return cambios

def recalcular_elo_por_season():
    """
    - Agrupa todos los partidos por season.
    - Para cada season (en orden cronológico), arranca Elo=1000 a todos,
      ordena los partidos de esa season por fecha y recalcula Elo.
    - Almacena en ranking_elo_por_season[season] el Elo final de cada jugador.
    - Almacena en elo_changes_por_partido[idx] la variación para cada jugador
      en el partido 'idx' (donde idx es el índice en la lista 'resultados').
    """
    ranking_elo_por_season.clear()
    elo_changes_por_partido.clear()

    # 1. Agrupar partidos por season
    from collections import defaultdict
    seasons_dict = defaultdict(list)
    for idx, partido in enumerate(resultados):
        season = partido["season"]
        seasons_dict[season].append((idx, partido))

    # 2. Ordenar las seasons por su número (Season 0, Season 1, Season 2, ...)
    def season_sort_key(s):
        if s == "Season 0":
            return 0
        else:
            try:
                return int(s.split()[1])
            except:
                return 9999
    sorted_seasons = sorted(seasons_dict.keys(), key=season_sort_key)

    # 3. Para cada season:
    for season in sorted_seasons:
        # Iniciamos todos los jugadores en 1000
        ranking_local = {j: 1000 for j in jugadores}
        # Ordenar los partidos de esta season por fecha
        lista_partidos = seasons_dict[season]
        # Cada elemento es (idx, partido)
        lista_partidos.sort(key=lambda x: x[1]["fecha"])

        # Procesar cada partido
        for (idx, p) in lista_partidos:
            cambios = actualizar_elo_sin_guardar(ranking_local, p)
            # Guardamos la variación de Elo de este partido
            elo_changes_por_partido[idx] = cambios

        # Guardar el ranking final de la season
        ranking_elo_por_season[season] = ranking_local.copy()

# -------------------------------------------------------------------
# 8. Mostrar Ranking Elo por Season (en pestañas)
# -------------------------------------------------------------------
def mostrar_ranking_elo():
    """
    Crea una ventana con un Notebook. Cada pestaña corresponde a una Season.
    En cada pestaña se muestra el Ranking Elo final de esa Season.
    """
    # Recalcular antes de mostrar
    recalcular_elo_por_season()

    ranking_window = tk.Toplevel()
    ranking_window.title("Ranking Elo (por Seasons)")
    ranking_window.geometry("800x600")

    notebook = ttk.Notebook(ranking_window)
    notebook.pack(expand=True, fill="both")

    # Ordenar las seasons para mostrar en orden
    def season_sort_key(s):
        if s == "Season 0":
            return 0
        else:
            try:
                return int(s.split()[1])
            except:
                return 9999
    sorted_seasons = sorted(ranking_elo_por_season.keys(), key=season_sort_key)

    # Para cada season, creamos un frame y un Treeview
    for season in sorted_seasons:
        frame = tk.Frame(notebook)
        notebook.add(frame, text=season)

        # Tomamos el ranking de esa season y lo ordenamos desc por Elo
        ranking_local = ranking_elo_por_season[season]
        ranking_ordenado = sorted(ranking_local.items(), key=lambda x: x[1], reverse=True)

        tree = ttk.Treeview(frame, columns=("Pos", "Jugador", "Elo"), show='headings')
        tree.heading("Pos", text="Posición")
        tree.heading("Jugador", text="Jugador")
        tree.heading("Elo", text="Elo")
        tree.column("Pos", anchor="center", width=80)
        tree.column("Jugador", anchor="center", width=150)
        tree.column("Elo", anchor="center", width=80)

        # Insertar filas
        pos = 1
        for (jug, elo) in ranking_ordenado:
            tree.insert("", tk.END, values=(pos, jug, elo))
            pos += 1

        tree.pack(expand=True, fill="both")

# -------------------------------------------------------------------
# 9. Mostrar Partidos (por Season) con columna Elo
# -------------------------------------------------------------------
def mostrar_partidos():
    """
    - Se crea un Notebook con pestañas (una por Season).
    - Cada pestaña muestra una tabla de partidos de esa Season.
    - Al filtrar por un jugador, se muestra en la columna "Elo" la variación
      que obtuvo ese jugador en ese partido.
    """
    # Asegurarnos de tener el Elo recalcado y los elo_changes_por_partido listos
    recalcular_elo_por_season()

    partidos_window = tk.Toplevel()
    partidos_window.title("Lista de Partidos")
    partidos_window.geometry("1000x600")

    style = ttk.Style(partidos_window)
    style.theme_use('clam')
    style.configure('Treeview', background='#E3F2FD', foreground='black',
                    rowheight=25, fieldbackground='#E3F2FD')
    style.configure('Treeview.Heading', background='#1E88E5',
                    foreground='white', font=('Helvetica', 10, 'bold'))

    filtro_frame = tk.Frame(partidos_window)
    filtro_frame.pack(pady=5)

    tk.Label(filtro_frame, text="Filtrar por Jugador:", font=('Helvetica', 12)).grid(row=0, column=0, padx=5)
    jugador_filtro_var = tk.StringVar(value="Todos")
    lista_jugadores_filtro = ["Todos"] + jugadores
    jugador_filtro_combobox = ttk.Combobox(filtro_frame, textvariable=jugador_filtro_var,
                                           values=lista_jugadores_filtro, state='readonly')
    jugador_filtro_combobox.grid(row=0, column=1, padx=5)

    notebook = ttk.Notebook(partidos_window)
    notebook.pack(expand=True, fill='both')

    # Agrupar partidos por season (ya tenemos la info en "resultados")
    from collections import defaultdict
    seasons_dict = defaultdict(list)
    for idx, r in enumerate(resultados):
        season = r["season"]
        seasons_dict[season].append((idx, r))

    # Ordenar seasons
    def season_sort_key(s):
        if s == "Season 0":
            return 0
        else:
            try:
                return int(s.split()[1])
            except:
                return 9999
    sorted_seasons = sorted(seasons_dict.keys(), key=season_sort_key)

    # Crear un Treeview por Season
    treeviews = {}
    columnas = ["Fecha", "Equipo 1", "Equipo 2", "Puntuaciones",
                "Ganador", "MVP", "Tie-breaks", "Lugar", "Elo"]
    for season in sorted_seasons:
        frame = tk.Frame(notebook)
        notebook.add(frame, text=season)
        tree = ttk.Treeview(frame, columns=columnas, show='headings')
        for col in columnas:
            tree.heading(col, text=col)
            tree.column(col, anchor='center', width=120, stretch=False)
        tree.pack(expand=True, fill='both')

        scrollbar_y = ttk.Scrollbar(frame, orient='vertical', command=tree.yview)
        scrollbar_y.pack(side='right', fill='y')
        tree.configure(yscrollcommand=scrollbar_y.set)

        treeviews[season] = tree

    def actualizar_partidos():
        filtro = jugador_filtro_var.get()

        for season in sorted_seasons:
            tree = treeviews[season]
            # Limpiar
            tree.delete(*tree.get_children())
            # Obtener los partidos de esa season
            lista_partidos = seasons_dict[season]
            # Ordenar por fecha
            lista_partidos.sort(key=lambda x: x[1]["fecha"])

            for (idx, r) in lista_partidos:
                fecha = r["fecha"]
                eq1_str = " & ".join(r["partido"][0])
                eq2_str = " & ".join(r["partido"][1])
                puntuaciones = "; ".join(r["puntuaciones"]) if r["puntuaciones"] else "N/A"
                ganador = " & ".join(r["ganador_partido"])
                mvp = r["mvp"]
                tie_breaks = r["tie_breaks"]
                lugar = r["lugar"]

                # Determinar si mostramos este partido
                if filtro == "Todos":
                    mostrar = True
                else:
                    jug_partido = list(r["partido"][0]) + list(r["partido"][1])
                    mostrar = (filtro in jug_partido)

                if mostrar:
                    # Elo difference
                    elo_val = ""
                    if filtro != "Todos":
                        # Mirar en elo_changes_por_partido[idx]
                        cambios = elo_changes_por_partido.get(idx, {})
                        if filtro in cambios:
                            diff = cambios[filtro]
                            # Por ejemplo: +10 o -8
                            if diff > 0:
                                elo_val = f"+{diff}"
                            else:
                                elo_val = f"{diff}"

                    tree.insert("", tk.END, values=(fecha, eq1_str, eq2_str,
                                                    puntuaciones, ganador, mvp,
                                                    tie_breaks, lugar, elo_val))

    # Evento al cambiar el filtro
    jugador_filtro_combobox.bind("<<ComboboxSelected>>", lambda e: actualizar_partidos())
    actualizar_partidos()

    ttk.Button(partidos_window, text="Cerrar", command=partidos_window.destroy).pack(pady=5)

# -------------------------------------------------------------------
# 10. Funciones de estadísticas, gráficos, etc. (opcionales)
# -------------------------------------------------------------------
def calcular_estadisticas(resultados_filtrar):
    """ Ejemplo de función de estadísticas. Puedes mantener la tuya. """
    lugares = ["Ibaiondo", "Bakh", "Otro"]
    estadisticas = {}
    for j in jugadores:
        estadisticas[j] = {
            "partidos_jugados": 0,
            "victorias": 0,
            "mvp": 0,
            "sets_jugados": 0,
            "sets_ganados": 0,
            "tie_breaks": 0,
            "primer_set_ganado": 0,
            "games_ganados": 0,
            "games_perdidos": 0,
            "victorias_por_lugar": {l: 0 for l in lugares}
        }
    for r in resultados_filtrar:
        equipo1, equipo2 = r["partido"]
        ganador = r["ganador_partido"]
        mvp = r["mvp"]
        lugar = r["lugar"]
        sets_jugados = len(r["puntuaciones"])
        for jug in equipo1 + equipo2:
            if jug not in estadisticas:
                estadisticas[jug] = {
                    "partidos_jugados": 0,
                    "victorias": 0,
                    "mvp": 0,
                    "sets_jugados": 0,
                    "sets_ganados": 0,
                    "tie_breaks": 0,
                    "primer_set_ganado": 0,
                    "games_ganados": 0,
                    "games_perdidos": 0,
                    "victorias_por_lugar": {l: 0 for l in lugares}
                }
        for jug in equipo1 + equipo2:
            estadisticas[jug]["partidos_jugados"] += 1
            estadisticas[jug]["sets_jugados"] += sets_jugados
        for jug in ganador:
            estadisticas[jug]["victorias"] += 1
            if lugar in estadisticas[jug]["victorias_por_lugar"]:
                estadisticas[jug]["victorias_por_lugar"][lugar] += 1
        for jug in r["ganador_primer_set"]:
            estadisticas[jug]["primer_set_ganado"] += 1
        for set_result in r["puntuaciones"]:
            if '(' in set_result:
                score_part, _ = set_result.split('(')
                s1, s2 = map(int, score_part.split('-'))
                tie_breaks_in_set = 1
            else:
                s1, s2 = map(int, set_result.split('-'))
                tie_breaks_in_set = 0
            for jug in equipo1 + equipo2:
                estadisticas[jug]["tie_breaks"] += tie_breaks_in_set
            if s1 > s2:
                for jug in equipo1:
                    estadisticas[jug]["sets_ganados"] += 1
            else:
                for jug in equipo2:
                    estadisticas[jug]["sets_ganados"] += 1
            for jug in equipo1:
                estadisticas[jug]["games_ganados"] += s1
                estadisticas[jug]["games_perdidos"] += s2
            for jug in equipo2:
                estadisticas[jug]["games_ganados"] += s2
                estadisticas[jug]["games_perdidos"] += s1
        if mvp in estadisticas:
            estadisticas[mvp]["mvp"] += 1
    # Calcular porcentajes
    for jug, st in estadisticas.items():
        pj = st["partidos_jugados"]
        if pj > 0:
            st["porcentaje_victorias"] = st["victorias"] / pj * 100
            st["porcentaje_primer_set"] = st["primer_set_ganado"] / pj * 100
        else:
            st["porcentaje_victorias"] = 0
            st["porcentaje_primer_set"] = 0
        st["diferencia_games"] = st["games_ganados"] - st["games_perdidos"]
    return estadisticas

def mostrar_grafico_jugadores():
    # Ejemplo
    messagebox.showinfo("Info", "Aquí iría el gráfico de jugadores...")

def mostrar_grafico_acumulado():
    # Ejemplo
    messagebox.showinfo("Info", "Aquí iría el gráfico acumulado...")

def mostrar_heatmap_sinergia():
    # Ejemplo
    messagebox.showinfo("Info", "Aquí iría el heatmap de sinergia...")

# -------------------------------------------------------------------
# 11. Interfaz principal
# -------------------------------------------------------------------
def crear_interfaz():
    root = tk.Tk()
    root.title("Registrar Resultado de Partido")
    root.geometry("900x700")

    style = ttk.Style(root)
    style.theme_use('clam')
    primary_color = '#1E88E5'
    background_color = '#E3F2FD'
    root.configure(bg=background_color)
    style.configure('TButton', font=('Segoe UI', 10), padding=5)
    style.configure('TLabel', font=('Segoe UI', 10))
    style.configure('TCombobox', font=('Segoe UI', 10))

    # Menú de navegación
    menu_bar = tk.Menu(root)
    navegacion_menu = tk.Menu(menu_bar, tearoff=0)
    navegacion_menu.add_command(label="Ranking Elo (Seasons)", command=mostrar_ranking_elo)
    navegacion_menu.add_command(label="Mostrar Partidos (Seasons)", command=mostrar_partidos)
    navegacion_menu.add_command(label="Gráfico Jugadores", command=mostrar_grafico_jugadores)
    navegacion_menu.add_command(label="Gráfico Acumulado", command=mostrar_grafico_acumulado)
    navegacion_menu.add_command(label="Heatmap Sinergia", command=mostrar_heatmap_sinergia)
    menu_bar.add_cascade(label="Navegación", menu=navegacion_menu)
    root.config(menu=menu_bar)

    # Campos del formulario
    tk.Label(root, text="Fecha del Partido (YYYY-mm-dd):", bg=background_color).grid(row=0, column=0, sticky='e')
    fecha_var = DateEntry(root, width=12, background='darkblue', foreground='white',
                          borderwidth=2, date_pattern='y-mm-dd')
    fecha_var.grid(row=0, column=1, pady=5, padx=5)

    tk.Label(root, text="Equipo 1:", font=('Helvetica', 12, 'bold'),
             bg=background_color).grid(row=1, column=0, sticky='e')
    equipo1_var = tk.StringVar()
    equipo1_combobox = ttk.Combobox(root, textvariable=equipo1_var, values=equipos_str)
    equipo1_combobox.grid(row=1, column=1, pady=5, padx=5)

    tk.Label(root, text="Equipo 2:", font=('Helvetica', 12, 'bold'),
             bg=background_color).grid(row=2, column=0, sticky='e')
    equipo2_var = tk.StringVar()
    equipo2_combobox = ttk.Combobox(root, textvariable=equipo2_var, values=equipos_str)
    equipo2_combobox.grid(row=2, column=1, pady=5, padx=5)

    tk.Label(root, text="Ganador 1er Set:", font=('Helvetica', 12, 'bold'),
             bg=background_color).grid(row=3, column=0, sticky='e')
    ganador_primer_set_var = tk.StringVar()
    ganador_primer_set_combobox = ttk.Combobox(root, textvariable=ganador_primer_set_var)
    ganador_primer_set_combobox.grid(row=3, column=1, pady=5, padx=5)

    tk.Label(root, text="Ganador Partido:", font=('Helvetica', 12, 'bold'),
             bg=background_color).grid(row=4, column=0, sticky='e')
    ganador_partido_var = tk.StringVar()
    ganador_partido_combobox = ttk.Combobox(root, textvariable=ganador_partido_var)
    ganador_partido_combobox.grid(row=4, column=1, pady=5, padx=5)

    tk.Label(root, text="MVP:", font=('Helvetica', 12, 'bold'),
             bg=background_color).grid(row=5, column=0, sticky='e')
    mvp_jugador = ttk.Combobox(root, values=jugadores)
    mvp_jugador.grid(row=5, column=1, pady=5, padx=5)

    # Sets
    set_resultados = {}
    tie_break_vars = {}
    tie_break_scores = {}

    def toggle_tiebreak_entry(num):
        if tie_break_vars[num].get():
            tie_break_scores[num].config(state='normal')
        else:
            tie_break_scores[num].delete(0, tk.END)
            tie_break_scores[num].config(state='disabled')

    for i in range(1, 4):
        tk.Label(root, text=f"Set {i} (ej: 6-4):", bg=background_color).grid(row=5+i, column=0, sticky='e')
        set_resultados[i] = ttk.Entry(root)
        set_resultados[i].grid(row=5+i, column=1, pady=5, padx=5)

        tie_break_vars[i] = tk.BooleanVar()
        cb = tk.Checkbutton(root, text="Tie-break", variable=tie_break_vars[i],
                            bg=background_color, command=lambda n=i: toggle_tiebreak_entry(n))
        cb.grid(row=5+i, column=2, padx=5)

        tk.Label(root, text=f"Puntuación Tie-break Set {i}:", bg=background_color).grid(row=5+i, column=3, sticky='e')
        tie_break_scores[i] = ttk.Entry(root, state='disabled')
        tie_break_scores[i].grid(row=5+i, column=4, pady=5, padx=5)

    tk.Label(root, text="Lugar del Partido:", bg=background_color).grid(row=9, column=0, sticky='e')
    lugar_var = tk.StringVar(value="Ibaiondo")
    lugar_menu = ttk.Combobox(root, textvariable=lugar_var, values=["Ibaiondo", "Bakh", "Otro"])
    lugar_menu.grid(row=9, column=1, pady=5, padx=5)

    # Al cambiar equipo1/equipo2, actualizar combos de ganador
    def actualizar_ganadores(*args):
        e1 = equipo1_var.get()
        e2 = equipo2_var.get()
        if e1 and e2:
            ganador_primer_set_combobox['values'] = [e1, e2]
            ganador_partido_combobox['values'] = [e1, e2]
        else:
            ganador_primer_set_combobox['values'] = []
            ganador_partido_combobox['values'] = []

    equipo1_var.trace("w", actualizar_ganadores)
    equipo2_var.trace("w", actualizar_ganadores)

    # Botón Registrar
    def registrar_partido():
        e1_str = equipo1_var.get()
        e2_str = equipo2_var.get()
        if not e1_str or not e2_str or e1_str == e2_str:
            messagebox.showerror("Error", "Equipos inválidos.")
            return
        e1 = equipo_str_a_pareja.get(e1_str)
        e2 = equipo_str_a_pareja.get(e2_str)
        if not e1 or not e2:
            messagebox.showerror("Error", "Equipos no válidos.")
            return

        g1set = ganador_primer_set_var.get()
        gpart = ganador_partido_var.get()
        if not g1set or not gpart:
            messagebox.showerror("Error", "Selecciona ganador de primer set y del partido.")
            return
        g1set_equip = e1 if g1set == e1_str else e2
        gpart_equip = e1 if gpart == e1_str else e2

        mvp = mvp_jugador.get()
        if not mvp:
            messagebox.showerror("Error", "Selecciona un MVP.")
            return

        fecha_dt = fecha_var.get_date()  # tkcalendar retorna datetime
        fecha_str = fecha_dt.strftime('%Y-%m-%d')
        season = obtener_season(fecha_str)

        # Sets
        puntuaciones = []
        tie_breaks_total = 0
        for i in range(1, 4):
            set_val = set_resultados[i].get()
            if set_val:
                try:
                    s1, s2 = map(int, set_val.split('-'))
                except:
                    messagebox.showerror("Error", f"Set {i} inválido. Usa formato n-n.")
                    return
                if tie_break_vars[i].get():
                    tb_score = tie_break_scores[i].get()
                    if not tb_score:
                        messagebox.showerror("Error", f"Falta puntaje tie-break en set {i}.")
                        return
                    set_str = f"{s1}-{s2}({tb_score})"
                    tie_breaks_total += 1
                else:
                    set_str = f"{s1}-{s2}"
                puntuaciones.append(set_str)

        resultado = {
            "partido": (e1, e2),
            "ganador_primer_set": g1set_equip,
            "ganador_partido": gpart_equip,
            "mvp": mvp,
            "puntuaciones": puntuaciones,
            "tie_breaks": tie_breaks_total,
            "lugar": lugar_var.get(),
            "fecha": fecha_str,
            "season": season
        }
        resultados.append(resultado)
        guardar_resultado_csv(resultado)
        messagebox.showinfo("OK", "Partido registrado.")

        # Limpiar campos
        equipo1_var.set("")
        equipo2_var.set("")
        ganador_primer_set_var.set("")
        ganador_partido_var.set("")
        mvp_jugador.set("")
        for i in range(1, 4):
            set_resultados[i].delete(0, tk.END)
            tie_break_vars[i].set(False)
            tie_break_scores[i].delete(0, tk.END)
            tie_break_scores[i].config(state='disabled')

    # Botones
    btn_frame = tk.Frame(root, bg=background_color)
    btn_frame.grid(row=10, columnspan=5, pady=10)
    tk.Button(btn_frame, text="Registrar Resultado",
              command=registrar_partido, bg=primary_color, fg='white').grid(row=0, column=0, padx=5)
    tk.Button(btn_frame, text="Mostrar Ranking (Seasons)",
              command=mostrar_ranking_elo, bg=primary_color, fg='white').grid(row=0, column=1, padx=5)
    tk.Button(btn_frame, text="Mostrar Partidos (Seasons)",
              command=mostrar_partidos, bg=primary_color, fg='white').grid(row=0, column=2, padx=5)

    def gestionar_jugadores():
        w = tk.Toplevel(root)
        w.title("Gestión de Jugadores")
        listbox = tk.Listbox(w)
        listbox.pack(side='left', fill='both', expand=True)
        scroll = ttk.Scrollbar(w, orient='vertical', command=listbox.yview)
        scroll.pack(side='left', fill='y')
        listbox.config(yscrollcommand=scroll.set)

        def refrescar():
            listbox.delete(0, tk.END)
            for jug in jugadores:
                listbox.insert(tk.END, jug)
        refrescar()

        def add_jug():
            name = simpledialog.askstring("Nuevo Jugador", "Nombre:")
            if name:
                name = name.strip()
                if name and name not in jugadores:
                    jugadores.append(name)
                    guardar_jugadores()
                    refrescar()
                    actualizar_datos_equipos()
                else:
                    messagebox.showerror("Error", "Jugador ya existe o inválido.")

        def edit_jug():
            sel = listbox.curselection()
            if not sel:
                return
            idx = sel[0]
            old_name = jugadores[idx]
            new_name = simpledialog.askstring("Editar Jugador", "Nuevo nombre:", initialvalue=old_name)
            if new_name:
                new_name = new_name.strip()
                if new_name and new_name not in jugadores:
                    jugadores[idx] = new_name
                    guardar_jugadores()
                    refrescar()
                    actualizar_datos_equipos()
                else:
                    messagebox.showerror("Error", "Jugador ya existe o inválido.")

        def del_jug():
            sel = listbox.curselection()
            if not sel:
                return
            idx = sel[0]
            jug = jugadores[idx]
            if messagebox.askyesno("Confirmar", f"¿Eliminar {jug}?"):
                jugadores.pop(idx)
                guardar_jugadores()
                refrescar()
                actualizar_datos_equipos()

        f_btn = tk.Frame(w)
        f_btn.pack(side='right', fill='y')
        tk.Button(f_btn, text="Agregar", command=add_jug).pack(pady=5)
        tk.Button(f_btn, text="Editar", command=edit_jug).pack(pady=5)
        tk.Button(f_btn, text="Eliminar", command=del_jug).pack(pady=5)

    tk.Button(btn_frame, text="Gestión de Jugadores",
              command=gestionar_jugadores, bg=primary_color, fg='white').grid(row=0, column=3, padx=5)

    # Cargar datos al inicio
    leer_jugadores()
    leer_resultados()
    actualizar_datos_equipos()

    root.mainloop()

# -------------------------------------------------------------------
# 12. Iniciar la app
# -------------------------------------------------------------------
if __name__ == "__main__":
    crear_interfaz()


In [14]:
!pyinstaller --onefile --windowed Padl.py


2340 INFO: PyInstaller: 6.11.1, contrib hooks: 2024.10
2341 INFO: Python: 3.11.5 (conda)
2351 INFO: Platform: Windows-10-10.0.22621-SP0
2351 INFO: Python environment: C:\Users\imontero\AppData\Local\anaconda3
2351 INFO: wrote C:\Users\imontero\Padl\Padl.spec
2359 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Scripts\\pyinstaller.exe',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\imontero\\Padl']
4943 INFO: Appending 'datas' from .spec
4943 

2254 INFO: PyInstaller: 6.11.1, contrib hooks: 2024.10
2254 INFO: Python: 3.11.5 (conda)
2258 INFO: Platform: Windows-10-10.0.22621-SP0
2260 INFO: Python environment: C:\Users\imontero\AppData\Local\anaconda3
2260 INFO: wrote C:\Users\imontero\Padl\Padl.spec
2264 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Scripts\\pyinstaller.exe',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\imontero\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\imontero\\Padl']
5082 INFO: checking Analysis
5103 INFO: Build